# PMF-GWR Based CNN-GNN-MLP

In [35]:
import numpy as np
import pandas as pd
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import cKDTree
from scipy.spatial import distance_matrix
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    LayerNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc

# Define the single buffer size to use for CNN patches
BUFFER_METERS = 500

# ==================== 1. Load Data & Preprocessing ==================== #
# Load the main dataset and the river sampling data.
original = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")
river_100.drop(columns="Source", inplace=True)

# Identify columns for feature engineering and prediction
drop_cols = ['Stations', 'River', 'Lat', 'Long', 'geometry']
numeric_cols = original.drop(columns=drop_cols).columns.drop('AsR')
pmf_features = ['CrR', 'NiR', 'CuR', 'AsR', 'CdR', 'PbR', 'MR', 'SandR', 'SiltR', 'ClayR', 'FeR']

# --- IMPUTATION FIX: Fill NaN values with 0 before further processing ---
original.fillna(0, inplace=True)
river_100.fillna(0, inplace=True)

# Split original data into train and test sets for the ensemble model.
np.random.seed(42)
train_orig = original.sample(10, random_state=42)
test_orig = original.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# Define the coordinates and target variables
coords_train = train_combined[['Long', 'Lat']].values
coords_test = test_orig[['Long', 'Lat']].values
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 2. Feature Engineering from Model 1 ==================== #

# --- 2.1 PMF (NMF) for Source Apportionment ---
nmf = NMF(n_components=3, init='random', random_state=42, max_iter=1000)
# Ensure data for NMF does not contain NaN or negative values
G_train = nmf.fit_transform(np.maximum(train_combined[pmf_features].values, 0))
F = nmf.components_
print("\nPMF Source Profiles (F):\n", pd.DataFrame(F, columns=pmf_features))

# --- 2.2 Fixed Geographically Weighted Regression (GWR) ---
def gaussian_kernel(d, bw):
    return np.exp(-(d**2) / (2 * bw**2))

def fixed_gwr(coords, factors, y, bw=0.5):
    """Performs a fixed bandwidth GWR using a Gaussian kernel."""
    n = len(coords)
    preds = np.zeros(n)
    X = np.hstack([np.ones((n, 1)), factors])
    for i in range(n):
        dist = np.linalg.norm(coords - coords[i], axis=1)
        W = np.diag(gaussian_kernel(dist, bw))
        try:
            beta = np.linalg.pinv(X.T @ W @ X) @ (X.T @ W @ y.reshape(-1, 1))
            preds[i] = (np.array([1] + list(factors[i])) @ beta).item()
        except np.linalg.LinAlgError:
            # Handle singular matrix by using a simpler model
            preds[i] = y.mean()
    return preds.reshape(-1, 1)

GWR_train = fixed_gwr(coords_train, G_train, y_train, bw=0.5)

# --- 2.3 Interpolate PMF factors for the test set ---
def idw_interpolation(known_coords, known_values, query_coords, power=2):
    """Performs IDW to interpolate values from known points to query points."""
    tree = cKDTree(known_coords)
    dists, idxs = tree.query(query_coords, k=4)
    dists[dists == 0] = 1e-10  # Avoid division by zero
    weights = 1 / (dists ** power)
    weights /= weights.sum(axis=1)[:, None]
    return np.sum(weights * known_values[idxs], axis=1)

G_test = np.column_stack([idw_interpolation(coords_train, G_train[:, i], coords_test) for i in range(G_train.shape[1])])

# --- 2.4 Apply GWR to the interpolated PMF factors for the test set ---
GWR_test = fixed_gwr(coords_test, G_test, y_test, bw=0.5)

# --- 2.5 Interaction Features ---
def create_interactions(pmf, gwr):
    """Creates interaction features between PMF factors and GWR predictions."""
    interactions = pd.DataFrame()
    for i in range(pmf.shape[1]):
        interactions[f"PMF{i}_GWR"] = pmf[:, i] * gwr.flatten()
    return interactions

train_interact = create_interactions(G_train, GWR_train)
test_interact = create_interactions(G_test, GWR_test)

# ==================== 3. Prepare GNN & MLP Input ==================== #
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

mlp_data_train_raw = pd.DataFrame(
    np.hstack([
        train_combined[numeric_cols].values,
        G_train,
        GWR_train,
        train_interact.values
    ]),
    columns=list(numeric_cols) + [f"PMF_Factor{i}" for i in range(G_train.shape[1])] + ["GWR_Adjusted"] + list(train_interact.columns)
)

mlp_data_test_raw = pd.DataFrame(
    np.hstack([
        test_orig[numeric_cols].values,
        G_test,
        GWR_test,
        test_interact.values
    ]),
    columns=list(numeric_cols) + [f"PMF_Factor{i}" for i in range(G_test.shape[1])] + ["GWR_Adjusted"] + list(test_interact.columns)
)

# --- IMPUTATION FIX: Fill NaN in raw MLP data before scaling ---
mlp_data_train_raw.fillna(0, inplace=True)
mlp_data_test_raw.fillna(0, inplace=True)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(mlp_data_train_raw)
mlp_test = scaler.transform(mlp_data_test_raw)

# ==================== 4. Collect ALL Rasters for CNN ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"\nUsing {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 5. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    for lon, lat in coords:
        channels = []
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    # --- NORMALIZATION FIX: Add a small epsilon to avoid division by zero ---
                    max_val = np.nanmax(arr)
                    if max_val != 0:
                        arr /= max_val + 1e-8 # Add epsilon for stability
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        return (batch_cnn, batch_mlp, batch_gnn), batch_y

# ==================== 6. Define Custom Transformer Layer ==================== #
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training=False):
        x = tf.expand_dims(inputs, axis=1)
        attn_output = self.att(x, x)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        out2 = self.layernorm2(out1 + ffn_output)
        
        return tf.squeeze(out2, axis=1)

# ==================== 7. Define the New Fusion Model ==================== #
def build_fusion_model(patch_shape, gnn_dim, mlp_dim):
    # CNN input
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    cnn_3x3 = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_3x3 = MaxPooling2D((2,2))(cnn_3x3)
    cnn_3x3 = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_3x3)
    cnn_3x3 = MaxPooling2D((2,2))(cnn_3x3)
    cnn_3x3 = Flatten()(cnn_3x3)

    cnn_5x5 = Conv2D(32, (5,5), activation="relu", padding="same")(cnn_input)
    cnn_5x5 = MaxPooling2D((2,2))(cnn_5x5)
    cnn_5x5 = Conv2D(64, (5,5), activation="relu", padding="same")(cnn_5x5)
    cnn_5x5 = MaxPooling2D((2,2))(cnn_5x5)
    cnn_5x5 = Flatten()(cnn_5x5)

    cnn_7x7 = Conv2D(32, (7,7), activation="relu", padding="same")(cnn_input)
    cnn_7x7 = MaxPooling2D((2,2))(cnn_7x7)
    cnn_7x7 = Conv2D(64, (7,7), activation="relu", padding="same")(cnn_7x7)
    cnn_7x7 = MaxPooling2D((2,2))(cnn_7x7)
    cnn_7x7 = Flatten()(cnn_7x7)

    cnn_combined = Concatenate(name="cnn_combined")([cnn_3x3, cnn_5x5, cnn_7x7])
    cnn_out = Dense(128, activation="relu", name="cnn_out")(cnn_combined)

    # MLP branch (for numerical site features)
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    m = Dense(64, activation="relu")(mlp_input)
    mlp_out = Dense(32, activation="relu", name="mlp_out")(m)

    # GNN branch (for spatial connectivity)
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    g = Dense(64, activation="relu")(gnn_input)
    gnn_out = Dense(32, activation="relu", name="gnn_out")(g)

    # Meta-learner (Transformer Block)
    pre_transformer_features = Concatenate()([cnn_out, mlp_out, gnn_out])
    
    # Calculate the new embedding dimension
    embed_dim = pre_transformer_features.shape[1]
    
    transformer_out = TransformerBlock(
        embed_dim=embed_dim,
        num_heads=4,
        ff_dim=256
    )(pre_transformer_features)
    
    # Final Fusion Layer
    f = Dense(128, activation="relu")(transformer_out)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

# Function to evaluate the model on the test set
def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn), verbose=0).flatten())
    
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        # --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
        y_pred[np.isnan(y_pred)] = 0 # Replace NaNs with 0
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

# ==================== 8. Run the Analysis ==================== #
print("\n" + "="*80)
print(f"Analyzing with Enhanced CNN–GNN–MLP Model ({BUFFER_METERS}m)")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_fusion_model(cnn_patch_shape, gnn_input_dim, mlp_input_dim)
model.summary()

# Create data generators
train_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

# Train Model
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# Evaluate
y_pred_train = model.predict(train_generator, verbose=0).flatten()
# --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
y_pred_train[np.isnan(y_pred_train)] = 0
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n✅ Enhanced CNN–GNN–MLP Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
y_pred_baseline[np.isnan(y_pred_baseline)] = 0
baseline_r2 = r2_score(y_test, y_pred_baseline)
print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test), verbose=0).flatten()
y_pred_cnn_ablated[np.isnan(y_pred_cnn_ablated)] = 0
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test), verbose=0).flatten()
y_pred_mlp_ablated[np.isnan(y_pred_mlp_ablated)] = 0
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated), verbose=0).flatten()
y_pred_gnn_ablated[np.isnan(y_pred_gnn_ablated)] = 0
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
for i, feature_name in enumerate(mlp_data_train_raw.columns):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test), verbose=0).flatten()
    y_pred_shuffled[np.isnan(y_pred_shuffled)] = 0
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# Garbage collect to free up memory
del model, history, train_generator
gc.collect()


PMF Source Profiles (F):
          CrR        NiR        CuR       AsR       CdR        PbR         MR  \
0   1.011394   0.736558   1.692247  0.281341  0.081750   1.820546   0.744546   
1   6.361288   2.934156   7.088633  1.445853  0.265187   5.471062   3.794235   
2  21.198226  13.542373  26.909206  5.147733  1.449371  21.855110  14.985165   

       SandR      SiltR      ClayR           FeR  
0   0.626923   0.857056   0.708029    811.577541  
1   4.450678   3.851341   2.797802   3334.739331  
2  16.238105  14.757198  12.760148  12485.904320  

Using 26 raster layers for CNN input.
  - bui.tif
  - ndsi.tif
  - savi.tif
  - ndbsi.tif
  - ui.tif
  - ndwi.tif
  - ndbi.tif
  - awei.tif
  - evi.tif
  - mndwi.tif
  - ndvi.tif
  - LULC2020.tif
  - LULC2021.tif
  - LULC2022.tif
  - LULC2019.tif
  - LULC2018.tif
  - LULC2017.tif
  - Pb_R.tif
  - ClayR.tif
  - SandR.tif
  - CdR.tif
  - CrR.tif
  - AsR.tif
  - SiltR.tif
  - CuR.tif
  - NiR.tif

Analyzing with Enhanced CNN–GNN–MLP Model (500m)


Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 100, 100,  │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 100, 100,  │     20,832 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 100, 100,  │     40,800 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_41    │ (None, 50, 50,    │          0 │ conv2d_46[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_43    │ (None, 50, 50,    │          0 │ conv2d_48[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_45    │ (None, 50, 50,    │          0 │ conv2d_50[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 50, 50,    │     18,496 │ max_pooling2d_41… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 50, 50,    │     51,264 │ max_pooling2d_43… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 50, 50,    │    100,416 │ max_pooling2d_45… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_42    │ (None, 25, 25,    │          0 │ conv2d_47[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_44    │ (None, 25, 25,    │          0 │ conv2d_49[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_46    │ (None, 25, 25,    │          0 │ conv2d_51[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_21          │ (None, 40000)     │          0 │ max_pooling2d_42… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_22          │ (None, 40000)     │          0 │ max_pooling2d_44… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_23          │ (None, 40000)     │          0 │ max_pooling2d_46… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 22)        │          0 │ -               

 Total params: 16,337,057 (62.32 MB)

 Trainable params: 16,337,057 (62.32 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 294ms/step - loss: 34723.8008 - val_loss: 23331.9414
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 280ms/step - loss: 22914.8848 - val_loss: 10303.9258
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 284ms/step - loss: 9632.6924 - val_loss: 4718.3516
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 281ms/step - loss: 6271.0171 - val_loss: 5185.2856
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 280ms/step - loss: 6240.5991 - val_loss: 4652.2095
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - loss: 5114.9873 - val_loss: 4708.8525
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - loss: 4945.1816 - val_loss: 4530.8457
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 279ms/step - loss: 5727.6230 - val_loss: 3625.7747
Epoch 9/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 283ms/step - loss: 2996.6450 - val_loss: 2689.4751
Epoch 10/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - loss: 3152.1204 - val_loss: 1318.2704
Epoch 11/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 282ms/step - loss: 14

11364

In [41]:
import numpy as np
import pandas as pd
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import cKDTree
from scipy.spatial import distance_matrix
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    LayerNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc
import pickle

# Define the single buffer size to use for CNN patches
BUFFER_METERS = 500

# ==================== 1. Load Data & Preprocessing ==================== #
# Load the main dataset and the river sampling data.
original = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")
river_100.drop(columns="Source", inplace=True)

# Identify columns for feature engineering and prediction
drop_cols = ['Stations', 'River', 'Lat', 'Long', 'geometry']
numeric_cols = original.drop(columns=drop_cols).columns.drop('AsR')
pmf_features = ['CrR', 'NiR', 'CuR', 'AsR', 'CdR', 'PbR', 'MR', 'SandR', 'SiltR', 'ClayR', 'FeR']

# --- IMPUTATION FIX: Fill NaN values with 0 before further processing ---
original.fillna(0, inplace=True)
river_100.fillna(0, inplace=True)

# Split original data into train and test sets for the ensemble model.
np.random.seed(42)
train_orig = original.sample(10, random_state=42)
test_orig = original.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# Define the coordinates and target variables
coords_train = train_combined[['Long', 'Lat']].values
coords_test = test_orig[['Long', 'Lat']].values
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 2. Feature Engineering from Model 1 ==================== #

# --- 2.1 PMF (NMF) for Source Apportionment ---
nmf = NMF(n_components=3, init='random', random_state=42, max_iter=1000)
# Ensure data for NMF does not contain NaN or negative values
G_train = nmf.fit_transform(np.maximum(train_combined[pmf_features].values, 0))
F = nmf.components_
print("\nPMF Source Profiles (F):\n", pd.DataFrame(F, columns=pmf_features))

# --- 2.2 Fixed Geographically Weighted Regression (GWR) ---
def gaussian_kernel(d, bw):
    return np.exp(-(d**2) / (2 * bw**2))

def fixed_gwr(coords, factors, y, bw=0.5):
    """Performs a fixed bandwidth GWR using a Gaussian kernel."""
    n = len(coords)
    preds = np.zeros(n)
    X = np.hstack([np.ones((n, 1)), factors])
    for i in range(n):
        dist = np.linalg.norm(coords - coords[i], axis=1)
        W = np.diag(gaussian_kernel(dist, bw))
        try:
            beta = np.linalg.pinv(X.T @ W @ X) @ (X.T @ W @ y.reshape(-1, 1))
            preds[i] = (np.array([1] + list(factors[i])) @ beta).item()
        except np.linalg.LinAlgError:
            # Handle singular matrix by using a simpler model
            preds[i] = y.mean()
    return preds.reshape(-1, 1)

GWR_train = fixed_gwr(coords_train, G_train, y_train, bw=0.5)

# --- 2.3 Interpolate PMF factors for the test set ---
def idw_interpolation(known_coords, known_values, query_coords, power=2):
    """Performs IDW to interpolate values from known points to query points."""
    tree = cKDTree(known_coords)
    dists, idxs = tree.query(query_coords, k=4)
    dists[dists == 0] = 1e-10  # Avoid division by zero
    weights = 1 / (dists ** power)
    weights /= weights.sum(axis=1)[:, None]
    return np.sum(weights * known_values[idxs], axis=1)

G_test = np.column_stack([idw_interpolation(coords_train, G_train[:, i], coords_test) for i in range(G_train.shape[1])])

# --- 2.4 Apply GWR to the interpolated PMF factors for the test set ---
GWR_test = fixed_gwr(coords_test, G_test, y_test, bw=0.5)

# --- 2.5 Interaction Features ---
def create_interactions(pmf, gwr):
    """Creates interaction features between PMF factors and GWR predictions."""
    interactions = pd.DataFrame()
    for i in range(pmf.shape[1]):
        interactions[f"PMF{i}_GWR"] = pmf[:, i] * gwr.flatten()
    return interactions

train_interact = create_interactions(G_train, GWR_train)
test_interact = create_interactions(G_test, GWR_test)

# ==================== 3. Prepare GNN & MLP Input ==================== #
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

mlp_data_train_raw = pd.DataFrame(
    np.hstack([
        train_combined[numeric_cols].values,
        G_train,
        GWR_train,
        train_interact.values
    ]),
    columns=list(numeric_cols) + [f"PMF_Factor{i}" for i in range(G_train.shape[1])] + ["GWR_Adjusted"] + list(train_interact.columns)
)

mlp_data_test_raw = pd.DataFrame(
    np.hstack([
        test_orig[numeric_cols].values,
        G_test,
        GWR_test,
        test_interact.values
    ]),
    columns=list(numeric_cols) + [f"PMF_Factor{i}" for i in range(G_test.shape[1])] + ["GWR_Adjusted"] + list(test_interact.columns)
)

# --- IMPUTATION FIX: Fill NaN in raw MLP data before scaling ---
mlp_data_train_raw.fillna(0, inplace=True)
mlp_data_test_raw.fillna(0, inplace=True)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(mlp_data_train_raw)
mlp_test = scaler.transform(mlp_data_test_raw)

# ==================== 4. Collect ALL Rasters for CNN ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"\nUsing {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 5. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    for lon, lat in coords:
        channels = []
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    # --- NORMALIZATION FIX: Add a small epsilon to avoid division by zero ---
                    max_val = np.nanmax(arr)
                    if max_val != 0:
                        arr /= max_val + 1e-8 # Add epsilon for stability
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        return (batch_cnn, batch_mlp, batch_gnn), batch_y

# ==================== 6. Define Custom Transformer Layer ==================== #
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training=False):
        x = tf.expand_dims(inputs, axis=1)
        attn_output = self.att(x, x)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        out2 = self.layernorm2(out1 + ffn_output)
        
        return tf.squeeze(out2, axis=1)

# ==================== 7. Define the New Fusion Model ==================== #
def build_fusion_model(patch_shape, gnn_dim, mlp_dim):
    # CNN input
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    cnn_3x3 = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_3x3 = MaxPooling2D((2,2))(cnn_3x3)
    cnn_3x3 = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_3x3)
    cnn_3x3 = MaxPooling2D((2,2))(cnn_3x3)
    cnn_3x3 = Flatten()(cnn_3x3)

    cnn_5x5 = Conv2D(32, (5,5), activation="relu", padding="same")(cnn_input)
    cnn_5x5 = MaxPooling2D((2,2))(cnn_5x5)
    cnn_5x5 = Conv2D(64, (5,5), activation="relu", padding="same")(cnn_5x5)
    cnn_5x5 = MaxPooling2D((2,2))(cnn_5x5)
    cnn_5x5 = Flatten()(cnn_5x5)

    cnn_7x7 = Conv2D(32, (7,7), activation="relu", padding="same")(cnn_input)
    cnn_7x7 = MaxPooling2D((2,2))(cnn_7x7)
    cnn_7x7 = Conv2D(64, (7,7), activation="relu", padding="same")(cnn_7x7)
    cnn_7x7 = MaxPooling2D((2,2))(cnn_7x7)
    cnn_7x7 = Flatten()(cnn_7x7)

    cnn_combined = Concatenate(name="cnn_combined")([cnn_3x3, cnn_5x5, cnn_7x7])
    cnn_out = Dense(128, activation="relu", name="cnn_out")(cnn_combined)

    # MLP branch (for numerical site features)
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    m = Dense(64, activation="relu")(mlp_input)
    mlp_out = Dense(32, activation="relu", name="mlp_out")(m)

    # GNN branch (for spatial connectivity)
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    g = Dense(64, activation="relu")(gnn_input)
    gnn_out = Dense(32, activation="relu", name="gnn_out")(g)

    # Meta-learner (Transformer Block)
    pre_transformer_features = Concatenate()([cnn_out, mlp_out, gnn_out])
    
    # Calculate the new embedding dimension
    embed_dim = pre_transformer_features.shape[1]
    
    transformer_out = TransformerBlock(
        embed_dim=embed_dim,
        num_heads=4,
        ff_dim=256
    )(pre_transformer_features)
    
    # Final Fusion Layer
    f = Dense(128, activation="relu")(transformer_out)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

# Function to evaluate the model on the test set
def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn), verbose=0).flatten())
    
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        # --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
        y_pred[np.isnan(y_pred)] = 0 # Replace NaNs with 0
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

# ==================== 8. Run the Analysis ==================== #
print("\n" + "="*80)
print(f"Analyzing with Enhanced CNN–GNN–MLP Model ({BUFFER_METERS}m)")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_fusion_model(cnn_patch_shape, gnn_input_dim, mlp_input_dim)
model.summary()

# Create data generators
train_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

# Train Model
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# Evaluate
y_pred_train = model.predict(train_generator, verbose=0).flatten()
# --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
y_pred_train[np.isnan(y_pred_train)] = 0
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n✅ Enhanced CNN–GNN–MLP Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
y_pred_baseline[np.isnan(y_pred_baseline)] = 0
baseline_r2 = r2_score(y_test, y_pred_baseline)
print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test), verbose=0).flatten()
y_pred_cnn_ablated[np.isnan(y_pred_cnn_ablated)] = 0
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test), verbose=0).flatten()
y_pred_mlp_ablated[np.isnan(y_pred_mlp_ablated)] = 0
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated), verbose=0).flatten()
y_pred_gnn_ablated[np.isnan(y_pred_gnn_ablated)] = 0
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
for i, feature_name in enumerate(mlp_data_train_raw.columns):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test), verbose=0).flatten()
    y_pred_shuffled[np.isnan(y_pred_shuffled)] = 0
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# ==================== 10. Save Model and Data for Reproducibility ==================== #
print("\n" + "="*80)
print("Saving Model, Data, and Feature Importance Results")
print("="*80)

# Create the single output directory
output_dir = "cnn_gnn_mlp_pg"
os.makedirs(output_dir, exist_ok=True)

# Save the trained model in the Keras native format
model_filename = os.path.join(output_dir, f"fusion_model_{BUFFER_METERS}m.keras")
model.save(model_filename)
print(f"✅ Model saved to '{model_filename}'")

# Save the training history using pickle
history_filename = os.path.join(output_dir, "training_history.pkl")
with open(history_filename, 'wb') as f:
    pickle.dump(history.history, f)
print(f"✅ Training history saved to '{history_filename}'")

# --- New: Save Feature Importance Results ---
feature_importance_results = {
    "mlp_feature_names": mlp_data_train_raw.columns.tolist(),
    "mlp_permutation_importance": mlp_feature_importance,
    "cnn_ablation_importance": importance_cnn,
    "mlp_ablation_importance": importance_mlp,
    "gnn_ablation_importance": importance_gnn
}
importance_filename = os.path.join(output_dir, "feature_importance.pkl")
with open(importance_filename, 'wb') as f:
    pickle.dump(feature_importance_results, f)
print(f"✅ Feature importance results saved to '{importance_filename}'")

# Save processed NumPy arrays for later use
np.savez_compressed(
    os.path.join(output_dir, "processed_train_data.npz"),
    coords=coords_train,
    mlp=mlp_train,
    y=y_train
)
np.savez_compressed(
    os.path.join(output_dir, "processed_test_data.npz"),
    coords=coords_test,
    mlp=mlp_test,
    y=y_test
)
np.savez_compressed(
    os.path.join(output_dir, "gnn_data.npz"),
    gnn_train=gnn_train,
    gnn_test=gnn_test
)
print(f"✅ Processed data arrays saved to '{output_dir}'")

# Save the raw dataframes to CSV for easy inspection
train_combined.to_csv(os.path.join(output_dir, "train_combined.csv"), index=False)
test_orig.to_csv(os.path.join(output_dir, "test_orig.csv"), index=False)
print(f"✅ Raw dataframes saved to '{output_dir}'")

# Garbage collect to free up memory
del model, history, train_generator
gc.collect()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 100, 100,  │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 100, 100,  │     20,832 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 100, 100,  │     40,800 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_38    │ (None, 50, 50,    │          0 │ conv2d_41[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_40    │ (None, 50, 50,    │          0 │ conv2d_43[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_42    │ (None, 50, 50,    │          0 │ conv2d_45[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 50, 50,    │     18,496 │ max_pooling2d_38… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 50, 50,    │     51,264 │ max_pooling2d_40… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 50, 50,    │    100,416 │ max_pooling2d_42… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_39    │ (None, 25, 25,    │          0 │ conv2d_42[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_41    │ (None, 25, 25,    │          0 │ conv2d_44[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_43    │ (None, 25, 25,    │          0 │ conv2d_46[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_21          │ (None, 40000)     │          0 │ max_pooling2d_39… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_22          │ (None, 40000)     │          0 │ max_pooling2d_41… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_23          │ (None, 40000)     │          0 │ max_pooling2d_43… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 22)        │          0 │ -               

 Total params: 16,337,057 (62.32 MB)

 Trainable params: 16,337,057 (62.32 MB)

 Non-trainable params: 0 (0.00 B)

15097

# CNN GNN MLP

In [42]:
import numpy as np
import pandas as pd
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc
import pickle

# Define the buffer size in meters
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")
river_100.drop(columns="Source", inplace=True)

drop_cols = ['Stations', 'River', 'Lat', 'Long', 'geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# --- IMPUTATION FIX: Fill NaN values with 0 before further processing ---
orig.fillna(0, inplace=True)
river_100.fillna(0, inplace=True)

# Train-test split
np.random.seed(42)
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    # --- NORMALIZATION FIX: Add a small epsilon to avoid division by zero ---
                    max_val = np.nanmax(arr)
                    if max_val != 0:
                        arr /= max_val + 1e-8 # Add epsilon for stability
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, batch_size=4, shuffle=True, buffer_meters=BUFFER_METERS, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        
        # Slice the GNN adjacency matrix for the current batch
        batch_gnn = self.gnn_data[batch_indices, :]

        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long', 'Lat']].values
coords_test = test_orig[['Long', 'Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
# --- IMPUTATION FIX: Fill NaN in raw MLP data before scaling ---
train_combined.fillna(0, inplace=True)
test_orig.fillna(0, inplace=True)
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define Enhanced CNN–GNN–MLP Model ==================== #
def build_fusion_model(patch_shape, gnn_dim, mlp_dim):
    # CNN branch (for raster data)
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    x = Conv2D(32, (3,3), activation="relu")(cnn_input)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)
    cnn_out = Dense(128, activation="relu", name="cnn_out")(x)

    # MLP branch (for numerical site features)
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    m = Dense(64, activation="relu")(mlp_input)
    mlp_out = Dense(32, activation="relu", name="mlp_out")(m)

    # GNN branch (for spatial connectivity)
    # The GNN input dimension is now the number of training samples
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    g = Dense(64, activation="relu")(gnn_input)
    gnn_out = Dense(32, activation="relu", name="gnn_out")(g)

    # Fusion Layer
    combined = Concatenate()([cnn_out, mlp_out, gnn_out])
    f = Dense(128, activation="relu")(combined)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

# We need to determine the final GNN input dimension for the model
# It's the total number of training samples
batch_size = 4
gnn_input_dim = len(coords_train)

# Helper function to get CNN patch shape from rasters
def get_cnn_patch_shape(raster_paths, buffer_meters):
    with rasterio.open(raster_paths[0]) as src:
        res_x, _ = src.res
        buffer_pixels = int(buffer_meters / res_x)
        return (2 * buffer_pixels, 2 * buffer_pixels, len(raster_paths))

cnn_patch_shape = get_cnn_patch_shape(raster_paths, BUFFER_METERS)
model = build_fusion_model(cnn_patch_shape, gnn_input_dim, mlp_train.shape[1])
model.summary()

# ==================== 6. Create Data Generators ==================== #
# We create a separate generator for the validation data.
train_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    batch_size=batch_size,
    shuffle=True,
    buffer_meters=BUFFER_METERS
)

# Function to evaluate the model on the test set
def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn), verbose=0).flatten())
    
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        # --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
        y_pred[np.isnan(y_pred)] = 0
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse


# ==================== 7. Train Model ==================== #
print("\n" + "="*80)
print(f"Analyzing with CNN–GNN–MLP Model ({BUFFER_METERS}m)")
print("="*80)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator # Using the same generator for validation for this example
)


# ==================== 8. Evaluate ==================== #
# Re-create a data generator without shuffling for evaluation on the training set
train_eval_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    batch_size=batch_size,
    shuffle=False,
    buffer_meters=BUFFER_METERS
)

y_pred_train = model.predict(train_eval_generator, verbose=0).flatten()
# --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
y_pred_train[np.isnan(y_pred_train)] = 0
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n✅ CNN–GNN–MLP Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
y_pred_baseline[np.isnan(y_pred_baseline)] = 0
baseline_r2 = r2_score(y_test, y_pred_baseline)
print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test), verbose=0).flatten()
y_pred_cnn_ablated[np.isnan(y_pred_cnn_ablated)] = 0
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test), verbose=0).flatten()
y_pred_mlp_ablated[np.isnan(y_pred_mlp_ablated)] = 0
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated), verbose=0).flatten()
y_pred_gnn_ablated[np.isnan(y_pred_gnn_ablated)] = 0
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
mlp_data_test_raw = test_orig[numeric_cols]
for i, feature_name in enumerate(mlp_data_test_raw.columns):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test), verbose=0).flatten()
    y_pred_shuffled[np.isnan(y_pred_shuffled)] = 0
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# ==================== 10. Save Model and Data for Reproducibility ==================== #
print("\n" + "="*80)
print("Saving Model, Data, and Feature Importance Results")
print("="*80)

# Create the single output directory
output_dir = "cnn_gnn_mlp"
os.makedirs(output_dir, exist_ok=True)

# Save the trained model in the Keras native format
model_filename = os.path.join(output_dir, f"fusion_model_{BUFFER_METERS}m.keras")
model.save(model_filename)
print(f"✅ Model saved to '{model_filename}'")

# Save the training history using pickle
history_filename = os.path.join(output_dir, "training_history.pkl")
with open(history_filename, 'wb') as f:
    pickle.dump(history.history, f)
print(f"✅ Training history saved to '{history_filename}'")

# --- New: Save Feature Importance Results ---
feature_importance_results = {
    "mlp_feature_names": test_orig[numeric_cols].columns.tolist(),
    "mlp_permutation_importance": mlp_feature_importance,
    "cnn_ablation_importance": importance_cnn,
    "mlp_ablation_importance": importance_mlp,
    "gnn_ablation_importance": importance_gnn
}
importance_filename = os.path.join(output_dir, "feature_importance.pkl")
with open(importance_filename, 'wb') as f:
    pickle.dump(feature_importance_results, f)
print(f"✅ Feature importance results saved to '{importance_filename}'")

# Save processed NumPy arrays for later use
np.savez_compressed(
    os.path.join(output_dir, "processed_train_data.npz"),
    coords=coords_train,
    mlp=mlp_train,
    y=y_train
)
np.savez_compressed(
    os.path.join(output_dir, "processed_test_data.npz"),
    coords=coords_test,
    mlp=mlp_test,
    y=y_test
)
np.savez_compressed(
    os.path.join(output_dir, "gnn_data.npz"),
    gnn_train=gnn_train,
    gnn_test=gnn_test
)
print(f"✅ Processed data arrays saved to '{output_dir}'")

# Save the raw dataframes to CSV for easy inspection
train_combined.to_csv(os.path.join(output_dir, "train_combined.csv"), index=False)
test_orig.to_csv(os.path.join(output_dir, "test_orig.csv"), index=False)
print(f"✅ Raw dataframes saved to '{output_dir}'")

# Garbage collect to free up memory
del model, history, train_generator
gc.collect()

Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 98, 98,    │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_44    │ (None, 49, 49,    │          0 │ conv2d_47[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 47, 47,    │     18,496 │ max_pooling2d_44… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_45    │ (None, 23, 23,    │          0 │ conv2d_48[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_24          │ (None, 33856)     │          0 │ max_pooling2d_45… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_149 (Dense)   │ (None, 64)        │      1,024 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_150 (Dense)   │ (None, 64)        │      7,104 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_out (Dense)     │ (None, 128)       │  4,333,696 │ flatten_24[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_out (Dense)     │ (None, 32)        │      2,080 │ dense_149[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_out (Dense)     │ (None, 32)        │      2,080 │ dense_150[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, 192)       │          0 │ cnn_out[0][0],    │
│ (Concatenate)       │                   │            │ mlp_out[0][0],    │
│                     │                   │            │ gnn_out[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_151 (Dense)   │ (None, 128)       │     24,704 │ concatenate_20[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_82          │ (None, 128)       │          0 │ dense_151[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_152 (Dense)   │ (None, 64)        │      8,256 │ dropout_82[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_152[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,405,025 (16.80 MB)

 Trainable params: 4,405,025 (16.80 MB)

 Non-trainable params: 0 (0.00 B)

21309

# CNN GAT MLP

In [44]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import pickle

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")
# Remove 'Source' column if it exists in river_100 dataframe
if 'Source' in river_100.columns:
    river_100.drop(columns="Source", inplace=True)

drop_cols = ['Stations', 'River', 'Lat', 'Long', 'geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# --- IMPUTATION FIX: Fill NaN values with 0 before further processing ---
orig.fillna(0, inplace=True)
river_100.fillna(0, inplace=True)

# Train-test split
np.random.seed(42)
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    # --- NORMALIZATION FIX: Add a small epsilon to avoid division by zero ---
                    max_val = np.nanmax(arr)
                    if max_val != 0:
                        arr /= max_val + 1e-8 # Add epsilon for stability
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        
        # Slice the GNN adjacency matrix for the current batch
        batch_gnn = self.gnn_data[batch_indices, :]

        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long', 'Lat']].values
coords_test = test_orig[['Long', 'Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
# --- IMPUTATION FIX: Fill NaN in raw MLP data before scaling ---
train_combined.fillna(0, inplace=True)
test_orig.fillna(0, inplace=True)
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define Enhanced CNN–GNN–MLP Model ==================== #
def build_fusion_model(patch_shape, gnn_dim, mlp_dim):
    # CNN branch (for raster data)
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    x = Conv2D(32, (3,3), activation="relu")(cnn_input)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)
    cnn_out = Dense(128, activation="relu", name="cnn_out")(x)

    # MLP branch (for numerical site features)
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    m = Dense(64, activation="relu")(mlp_input)
    mlp_out = Dense(32, activation="relu", name="mlp_out")(m)

    # GNN branch (for spatial connectivity)
    # The GNN input dimension is now the number of training samples
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    g = Dense(64, activation="relu")(gnn_input)
    gnn_out = Dense(32, activation="relu", name="gnn_out")(g)

    # Fusion Layer
    combined = Concatenate()([cnn_out, mlp_out, gnn_out])
    f = Dense(128, activation="relu")(combined)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

# We need to determine the final GNN input dimension for the model
batch_size = 4
gnn_input_dim = len(coords_train)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_fusion_model(cnn_patch_shape, gnn_input_dim, mlp_train.shape[1])
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn), verbose=0).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        # --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
        y_pred[np.isnan(y_pred)] = 0
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

# ==================== 7. Train Model ==================== #
print("\n" + "="*80)
print(f"Analyzing with CNN–GAT–MLP Model ({BUFFER_METERS}m)")
print("="*80)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate ==================== #
train_eval_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=False
)

y_pred_train = model.predict(train_eval_generator, verbose=0).flatten()
# --- NaN FIX: Ensure y_pred has no NaNs before calculating metrics ---
y_pred_train[np.isnan(y_pred_train)] = 0
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n✅ CNN–GAT–MLP Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
y_pred_baseline[np.isnan(y_pred_baseline)] = 0
baseline_r2 = r2_score(y_test, y_pred_baseline)
print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test), verbose=0).flatten()
y_pred_cnn_ablated[np.isnan(y_pred_cnn_ablated)] = 0
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test), verbose=0).flatten()
y_pred_mlp_ablated[np.isnan(y_pred_mlp_ablated)] = 0
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated), verbose=0).flatten()
y_pred_gnn_ablated[np.isnan(y_pred_gnn_ablated)] = 0
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
mlp_data_test_raw = test_orig[numeric_cols]
for i, feature_name in enumerate(mlp_data_test_raw.columns):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test), verbose=0).flatten()
    y_pred_shuffled[np.isnan(y_pred_shuffled)] = 0
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# ==================== 10. Save Model and Data for Reproducibility ==================== #
print("\n" + "="*80)
print("Saving Model, Data, and Feature Importance Results")
print("="*80)

# Create the single output directory
output_dir = "cnn_gat_mlp"
os.makedirs(output_dir, exist_ok=True)

# Save the trained model in the Keras native format
model_filename = os.path.join(output_dir, f"fusion_model_{BUFFER_METERS}m.keras")
model.save(model_filename)
print(f"✅ Model saved to '{model_filename}'")

# Save the training history using pickle
history_filename = os.path.join(output_dir, "training_history.pkl")
with open(history_filename, 'wb') as f:
    pickle.dump(history.history, f)
print(f"✅ Training history saved to '{history_filename}'")

# --- New: Save Feature Importance Results ---
feature_importance_results = {
    "mlp_feature_names": test_orig[numeric_cols].columns.tolist(),
    "mlp_permutation_importance": mlp_feature_importance,
    "cnn_ablation_importance": importance_cnn,
    "mlp_ablation_importance": importance_mlp,
    "gnn_ablation_importance": importance_gnn
}
importance_filename = os.path.join(output_dir, "feature_importance.pkl")
with open(importance_filename, 'wb') as f:
    pickle.dump(feature_importance_results, f)
print(f"✅ Feature importance results saved to '{importance_filename}'")

# Save processed NumPy arrays for later use
np.savez_compressed(
    os.path.join(output_dir, "processed_train_data.npz"),
    coords=coords_train,
    mlp=mlp_train,
    y=y_train
)
np.savez_compressed(
    os.path.join(output_dir, "processed_test_data.npz"),
    coords=coords_test,
    mlp=mlp_test,
    y=y_test
)
np.savez_compressed(
    os.path.join(output_dir, "gnn_data.npz"),
    gnn_train=gnn_train,
    gnn_test=gnn_test
)
print(f"✅ Processed data arrays saved to '{output_dir}'")

# Save the raw dataframes to CSV for easy inspection
train_combined.to_csv(os.path.join(output_dir, "train_combined.csv"), index=False)
test_orig.to_csv(os.path.join(output_dir, "test_orig.csv"), index=False)
print(f"✅ Raw dataframes saved to '{output_dir}'")

# Garbage collect to free up memory
del model, history, train_generator
gc.collect()


Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 98, 98,    │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_46    │ (None, 49, 49,    │          0 │ conv2d_49[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 47, 47,    │     18,496 │ max_pooling2d_46… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_47    │ (None, 23, 23,    │          0 │ conv2d_50[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_25          │ (None, 33856)     │          0 │ max_pooling2d_47… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_153 (Dense)   │ (None, 64)        │      1,024 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_154 (Dense)   │ (None, 64)        │      7,104 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_out (Dense)     │ (None, 128)       │  4,333,696 │ flatten_25[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_out (Dense)     │ (None, 32)        │      2,080 │ dense_153[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_out (Dense)     │ (None, 32)        │      2,080 │ dense_154[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, 192)       │          0 │ cnn_out[0][0],    │
│ (Concatenate)       │                   │            │ mlp_out[0][0],    │
│                     │                   │            │ gnn_out[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_155 (Dense)   │ (None, 128)       │     24,704 │ concatenate_21[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_83          │ (None, 128)       │          0 │ dense_155[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_156 (Dense)   │ (None, 64)        │      8,256 │ dropout_83[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_156[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,405,025 (16.80 MB)

 Trainable params: 4,405,025 (16.80 MB)

 Non-trainable params: 0 (0.00 B)

12513

# Mixture of Experts (MoE) Ensemble

```
[Expert 1: CNN] ┐
[Expert 2: GNN] ├── Gating Network (softmax weights) → Weighted Sum → Output
[Expert 3: MLP] ┘

```

In [45]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    LayerNormalization,
    Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import pickle # For saving and loading the scaler and feature importance results
import json # For saving the feature importance results

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
# NOTE: The data loading logic remains the same as it provides the inputs
# required for the new model architecture.
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

# Define the columns to drop and the numeric columns to use for MLP
drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Ensure there are no NaNs in the numeric columns before proceeding
orig[numeric_cols] = orig[numeric_cols].fillna(0)
river_100[numeric_cols] = river_100[numeric_cols].fillna(0)

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("   -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    
    This version includes robust NaN handling to prevent model training errors.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    
                    # Corrected logic: Convert any NaNs to a numerical value, e.g., 0,
                    # to prevent them from propagating through the model.
                    arr = np.nan_to_num(arr, nan=0.0)
                    
                    arr = arr.astype(np.float32)

                    # Get the maximum value, but check if it's a valid number and > 0.
                    # This prevents division by zero if the patch is all zeros.
                    max_val = np.nanmax(arr)
                    if np.isfinite(max_val) and max_val > 0:
                        arr /= max_val
                        
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        
        # Slice the GNN adjacency matrix for the current batch
        batch_gnn = self.gnn_data[batch_indices, :]

        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y


# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values

# We now split the training data into a training and validation set
train_split, val_split = train_test_split(train_combined, test_size=0.2, random_state=42)

coords_train_split = train_split[['Long','Lat']].values
coords_val_split = val_split[['Long','Lat']].values

dist_mat_train_split = distance_matrix(coords_train_split, coords_train_split)
gnn_train_split = np.exp(-dist_mat_train_split/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train_split)
gnn_test = np.exp(-dist_mat_test_train/10)
dist_mat_val_train = distance_matrix(coords_val_split, coords_train_split)
gnn_val_split = np.exp(-dist_mat_val_train/10)


scaler = StandardScaler()
mlp_train_split = scaler.fit_transform(train_split[numeric_cols])
mlp_val_split = scaler.transform(val_split[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train_split = train_split['RI'].values
y_val_split = val_split['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define the Mixture of Experts Model ==================== #
def build_moe_model(patch_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Expert 1: CNN Branch ---
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch_flattened = Flatten()(cnn_branch)
    cnn_branch_dense = Dense(128, activation="relu")(cnn_branch_flattened)
    # The CNN expert's final prediction
    cnn_expert_out = Dense(1, activation="linear", name="cnn_expert_out")(cnn_branch_dense)

    # --- Expert 2: MLP Branch ---
    mlp_branch = Dense(64, activation="relu")(mlp_input)
    mlp_branch = Dense(32, activation="relu")(mlp_branch)
    # The MLP expert's final prediction
    mlp_expert_out = Dense(1, activation="linear", name="mlp_expert_out")(mlp_branch)

    # --- Expert 3: GNN Branch ---
    gnn_branch = Dense(64, activation="relu")(gnn_input)
    gnn_branch = Dense(32, activation="relu")(gnn_branch)
    # The GNN expert's final prediction
    gnn_expert_out = Dense(1, activation="linear", name="gnn_expert_out")(gnn_branch)

    # --- Gating Network ---
    # The gating network needs features from all inputs to make its decision.
    # We use the outputs of the dense layers before the final predictions as features.
    gate_input = Concatenate()([cnn_branch_dense, mlp_branch, gnn_branch])
    gate_network = Dense(64, activation="relu")(gate_input)
    gate_network = Dense(32, activation="relu")(gate_network)
    # The output is a set of weights for each expert (summing to 1 via softmax)
    gate_weights = Dense(3, activation="softmax", name="gate_weights")(gate_network)

    # --- Combine Experts and Gating Network ---
    # Stack the predictions from each expert.
    # The shape will be (batch_size, 3)
    experts_stack = Concatenate(axis=1, name="experts_stack")([cnn_expert_out, mlp_expert_out, gnn_expert_out])
    
    # Perform the weighted sum.
    # This is done using a Lambda layer which takes the experts' outputs and
    # the gating network's weights, and computes the dot product for each sample.
    final_output = Lambda(lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="final_output")([experts_stack, gate_weights])

    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=final_output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse


# ==================== Run the Analysis ==================== #
print("\n" + "="*80)
print(f"Analyzing for BUFFER_METERS = {BUFFER_METERS}m")
print("="*80)

batch_size = 4
gnn_input_dim = len(train_split)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_moe_model(cnn_patch_shape, gnn_input_dim, mlp_train_split.shape[1])
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train_split,
    mlp_data=mlp_train_split,
    gnn_data=gnn_train_split,
    y=y_train_split,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

validation_generator = DataGenerator(
    coords=coords_val_split,
    mlp_data=mlp_val_split,
    gnn_data=gnn_val_split,
    y=y_val_split,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=False # No need to shuffle validation data
)


# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=validation_generator
)

# ==================== 8. Evaluate ==================== #
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train_split[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train_split[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size)

print(f"\n Mixture of Experts Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 9.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
baseline_r2 = r2_score(y_test, y_pred_baseline)

print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test)).flatten()
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test)).flatten()
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated)).flatten()
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 9.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
for i, feature_name in enumerate(numeric_cols):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test)).flatten()
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# Garbage collect to free up memory
del model, history, train_generator, validation_generator
gc.collect()

# ==================== 10. Save Model and Results ==================== #
print("\n" + "="*80)
print("Saving Model and Analysis Results...")
print("="*80)

# Rebuild and re-train the model to ensure it's in a savable state.
# This is a good practice to avoid issues with saving during a live session.
# First, let's get the necessary dimensions again.
batch_size = 4
gnn_input_dim = len(train_split)
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

final_model = build_moe_model(cnn_patch_shape, gnn_input_dim, mlp_train_split.shape[1])
# Re-fit the model on the full training and validation data
final_model.fit(
    DataGenerator(coords=coords_train_split, mlp_data=mlp_train_split, gnn_data=gnn_train_split, y=y_train_split, raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size),
    epochs=2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    validation_data=DataGenerator(coords=coords_val_split, mlp_data=mlp_val_split, gnn_data=gnn_val_split, y=y_val_split, raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=False)
)

output_dir = "mixture_of_experts"
os.makedirs(output_dir, exist_ok=True)

# 10.1 Save the trained Keras model
model_name = "mixture_of_experts_model"
final_model.save(f'{output_dir}/{model_name}.keras')
print(f"Saved trained model to {output_dir}/{model_name}.keras")

# 10.2 Save the StandardScaler object
# Using pickle.dump for serialization
with open(f'{output_dir}/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"Saved StandardScaler object to {output_dir}/scaler.pkl")

# 10.3 Combine and save all feature importance results into a single file
feature_importance_results = {
    "combined_branch_importance": {
        "CNN_Importance_R2_drop": importance_cnn,
        "MLP_Importance_R2_drop": importance_mlp,
        "GNN_Importance_R2_drop": importance_gnn
    },
    "mlp_permutation_importance": mlp_feature_importance
}

# Using pickle.dump for serialization
with open(f'{output_dir}/feature_importance.pkl', 'wb') as f:
    pickle.dump(feature_importance_results, f)
print(f"Saved all feature importance results to {output_dir}/feature_importance.pkl")

# 10.4 Save the preprocessed data arrays for future use
np.save(f'{output_dir}/coords_train.npy', coords_train)
np.save(f'{output_dir}/mlp_train.npy', mlp_train_split)
np.save(f'{output_dir}/gnn_train.npy', gnn_train_split)
np.save(f'{output_dir}/y_train.npy', y_train_split)
np.save(f'{output_dir}/coords_test.npy', coords_test)
np.save(f'{output_dir}/mlp_test.npy', mlp_test)
np.save(f'{output_dir}/gnn_test.npy', gnn_test)
np.save(f'{output_dir}/y_test.npy', y_test)
print("Saved preprocessed data arrays to .npy files")

print("\nAll requested artifacts have been saved successfully.")

Model: "functional_40"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 100, 100,  │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_48    │ (None, 50, 50,    │          0 │ conv2d_51[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_52 (Conv2D)  │ (None, 50, 50,    │     18,496 │ max_pooling2d_48… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_49    │ (None, 25, 25,    │          0 │ conv2d_52[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 88)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_26          │ (None, 40000)     │          0 │ max_pooling2d_49… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_158 (Dense)   │ (None, 64)        │      1,024 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_160 (Dense)   │ (None, 64)        │      5,696 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_157 (Dense)   │ (None, 128)       │  5,120,128 │ flatten_26[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_159 (Dense)   │ (None, 32)        │      2,080 │ dense_158[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_161 (Dense)   │ (None, 32)        │      2,080 │ dense_160[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_22      │ (None, 192)       │          0 │ dense_157[0][0],  │
│ (Concatenate)       │                   │            │ dense_159[0][0],  │
│                     │                   │            │ dense_161[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_162 (Dense)   │ (None, 64)        │     12,352 │ concatenate_22[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_expert_out      │ (None, 1)         │        129 │ dense_157[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_expert_out      │ (None, 1)         │         33 │ dense_159[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_expert_out      │ (None, 1)         │         33 │ dense_161[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 5,171,750 (19.73 MB)

 Trainable params: 5,171,750 (19.73 MB)

 Non-trainable params: 0 (0.00 B)

# Dual Attention Ensemble

In [46]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    Lambda,
    GlobalAveragePooling2D,
    Reshape,
    Multiply
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO # To capture print output
import pickle # For saving dictionaries and other objects

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
# NOTE: The data loading logic remains the same.
# Replace with your actual data paths if needed
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    if np.nanmax(arr) != 0:
                        arr /= np.nanmax(arr)
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        
        # Slice the GNN adjacency matrix for the current batch
        batch_gnn = self.gnn_data[batch_indices, :]

        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        # Return a tuple of inputs and the target, which Keras expects
        return (batch_cnn, batch_mlp, batch_gnn), batch_y


# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define Custom Attention Layers ==================== #

class SpatialAttention(Layer):
    """
    A custom layer to apply spatial attention to a feature map.
    It generates a spatial attention map and multiplies it with the input.
    """
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.conv1 = Conv2D(1, (1, 1), activation='sigmoid')

    def call(self, inputs):
        # Squeeze the channels and generate a 2D attention map
        attention_map = self.conv1(inputs)
        # Multiply the input feature map by the attention map
        return Multiply()([inputs, attention_map])

class FeatureAttention(Layer):
    """
    A custom layer to apply feature-wise attention.
    It learns a weight for each feature channel and multiplies it with the input.
    Inspired by Squeeze-and-Excitation networks.
    """
    def __init__(self, reduction_ratio=16, **kwargs):
        super(FeatureAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        super(FeatureAttention, self).build(input_shape)
        if len(input_shape) == 4: # CNN output
            self.avg_pool = GlobalAveragePooling2D()
            self.dense1 = Dense(units=input_shape[-1] // self.reduction_ratio, activation='relu')
            self.dense2 = Dense(units=input_shape[-1], activation='sigmoid')
            self.reshape_output = Reshape((1, 1, input_shape[-1]))
        else: # MLP or GNN output
            self.dense1 = Dense(units=input_shape[-1] // self.reduction_ratio, activation='relu')
            self.dense2 = Dense(units=input_shape[-1], activation='sigmoid')

    def call(self, inputs):
        if len(inputs.shape) == 4: # CNN branch
            x = self.avg_pool(inputs)
            x = self.dense1(x)
            x = self.dense2(x)
            x = self.reshape_output(x)
        else: # MLP or GNN branch
            x = self.dense1(inputs)
            x = self.dense2(x)
            
        return Multiply()([inputs, x])

# ==================== 6. Define the Dual Attention Model ==================== #
def build_dual_attention_model(patch_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- CNN Branch with Spatial and Feature Attention ---
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    
    # Spatial Attention
    cnn_spatial_attn = SpatialAttention()(cnn_branch)
    
    # Feature Attention
    cnn_feature_attn = FeatureAttention()(cnn_spatial_attn)
    
    # Flatten and get embedding
    cnn_embedding = Flatten()(cnn_feature_attn)
    cnn_embedding = Dense(128, activation="relu", name="cnn_embedding")(cnn_embedding)

    # --- MLP Branch with Embedding ---
    mlp_embedding = Dense(64, activation="relu")(mlp_input)
    mlp_embedding = Dense(32, activation="relu", name="mlp_embedding")(mlp_embedding)

    # --- GNN Branch with Feature Attention and Embedding ---
    gnn_branch = Dense(64, activation="relu")(gnn_input)
    
    # Feature Attention
    gnn_feature_attn = FeatureAttention()(gnn_branch)
    gnn_embedding = Dense(32, activation="relu", name="gnn_embedding")(gnn_feature_attn)

    # --- Attention Fusion ---
    # Concatenate all embeddings
    combined_embedding = Concatenate(name="combined_embedding")([cnn_embedding, mlp_embedding, gnn_embedding])
    
    # Final dense layers for prediction
    f = Dense(128, activation="relu")(combined_embedding)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, coords_test, mlp_test, gnn_test_matrix, y_test, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        
        batch_gnn = gnn_test_matrix[i:i+batch_size, :]
        batch_y = y_test[i:i+batch_size]

        batch_cnn = extract_patch_for_generator(
            batch_coords,
            raster_paths,
            buffer_pixels_x,
            buffer_pixels_y,
            patch_width,
            patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

# ==================== Run the Analysis ==================== #
# Capture all print statements to a string
old_stdout = sys.stdout
sys.stdout = captured_output = StringIO()

print("\n" + "="*80)
print(f"Analyzing for BUFFER_METERS = {BUFFER_METERS}m")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

model = build_dual_attention_model(cnn_patch_shape, gnn_input_dim, mlp_train.shape[1])
model.summary(print_fn=lambda x: captured_output.write(x + '\n')) # Capture model summary

# ==================== 7. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train,
    mlp_data=mlp_train,
    gnn_data=gnn_train,
    y=y_train,
    raster_paths=raster_paths,
    buffer_meters=BUFFER_METERS,
    batch_size=batch_size,
    shuffle=True
)

# ==================== 8. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

print("\nStarting model training...")
history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)
print("Training complete.")

# ==================== 9. Evaluate ==================== #
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

# Get test predictions for saving as .npy
y_pred_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, return_preds=True)
r2_test = r2_score(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\n Dual Attention Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 10. Feature Importance Analysis ==================== #
print("\n" + "-"*50)
print(f"Feature Importance Analysis for {BUFFER_METERS}m")
print("-"*50)

# --- 10.1 Combined Feature Importance (by Model Branch) ---
y_pred_baseline = y_pred_test
baseline_r2 = r2_test
print(f"\nBaseline Performance on Test Set: R² = {baseline_r2:.4f}")

# Ablate CNN branch
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    buffer_pixels_y = int(BUFFER_METERS / res_y)
    patch_width = 2 * buffer_pixels_x
    patch_height = 2 * buffer_pixels_y

cnn_test_ablated = np.zeros_like(extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
))
y_pred_cnn_ablated = model.predict((cnn_test_ablated, mlp_test, gnn_test)).flatten()
r2_cnn_ablated = r2_score(y_test, y_pred_cnn_ablated)
importance_cnn = baseline_r2 - r2_cnn_ablated

# Ablate MLP branch
mlp_test_ablated = np.zeros_like(mlp_test)
y_pred_mlp_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test_ablated, gnn_test)).flatten()
r2_mlp_ablated = r2_score(y_test, y_pred_mlp_ablated)
importance_mlp = baseline_r2 - r2_mlp_ablated

# Ablate GNN branch
gnn_test_ablated = np.zeros_like(gnn_test)
y_pred_gnn_ablated = model.predict((extract_patch_for_generator(
    coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
), mlp_test, gnn_test_ablated)).flatten()
r2_gnn_ablated = r2_score(y_test, y_pred_gnn_ablated)
importance_gnn = baseline_r2 - r2_gnn_ablated

print("\n--- Combined Feature Importance (by Model Branch) ---")
print(f"CNN Branch Importance (R² drop): {importance_cnn:.4f}")
print(f"MLP Branch Importance (R² drop): {importance_mlp:.4f}")
print(f"GNN Branch Importance (R² drop): {importance_gnn:.4f}")

# --- 10.2 MLP Feature Importance (Permutation-based) ---
mlp_feature_importance = {}
for i, feature_name in enumerate(numeric_cols):
    mlp_test_shuffled = np.copy(mlp_test)
    np.random.shuffle(mlp_test_shuffled[:, i])
    
    y_pred_shuffled = model.predict((extract_patch_for_generator(
        coords_test, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
    ), mlp_test_shuffled, gnn_test)).flatten()
    r2_shuffled = r2_score(y_test, y_pred_shuffled)
    
    importance = baseline_r2 - r2_shuffled
    mlp_feature_importance[feature_name] = importance

print("\n--- MLP Feature Importance (Permutation-based) ---")
sorted_importance = sorted(mlp_feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_importance:
    print(f"{feature:<20}: {importance:.4f}")
    
# Garbage collect to free up memory
del history, train_generator
gc.collect()

# ==================== 11. Save all info to a folder ==================== #
# Restore standard output
sys.stdout = old_stdout
printed_output = captured_output.getvalue()

output_folder = "dual_attention_analysis"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save the trained model in the .keras format
model_path = os.path.join(output_folder, "dual_attention.keras")
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the MLP feature importance to a .pkl file
mlp_importance_path = os.path.join(output_folder, "mlp_feature_importance.pkl")
with open(mlp_importance_path, 'wb') as f:
    pickle.dump(mlp_feature_importance, f)
print(f"MLP feature importance saved to: {mlp_importance_path}")

# Save all relevant data to .npy files
np.save(os.path.join(output_folder, "coords_train.npy"), coords_train)
np.save(os.path.join(output_folder, "coords_test.npy"), coords_test)
np.save(os.path.join(output_folder, "mlp_train.npy"), mlp_train)
np.save(os.path.join(output_folder, "mlp_test.npy"), mlp_test)
np.save(os.path.join(output_folder, "gnn_train.npy"), gnn_train)
np.save(os.path.join(output_folder, "gnn_test.npy"), gnn_test)
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred_train.npy"), y_pred_train)
np.save(os.path.join(output_folder, "y_pred_test.npy"), y_pred_test)
print(f"Coordinates, scaled data, GNN matrices, and labels/predictions saved as .npy files.")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

print("All information successfully saved.")

# Stacked Ensemble

In [47]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Concatenate,
    Dropout,
    Layer,
    Lambda,
    GlobalAveragePooling2D,
    Reshape,
    Multiply
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
# NOTE: The data loading logic remains the same.
# Replace with your actual data paths if needed
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    if np.nanmax(arr) != 0:
                        arr /= np.nanmax(arr)
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )

        return (batch_cnn, batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define Base Models ==================== #
def build_cnn_mlp_model(patch_shape, mlp_dim):
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")

    # CNN branch
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_embedding = Flatten()(cnn_branch)

    # MLP branch
    mlp_embedding = Dense(64, activation="relu")(mlp_input)
    mlp_embedding = Dense(32, activation="relu")(mlp_embedding)

    # Combine
    combined = Concatenate()([cnn_embedding, mlp_embedding])
    f = Dense(128, activation="relu")(combined)
    output = Dense(1, activation="linear", name="cnn_mlp_output")(f)
    
    model = Model(inputs=[cnn_input, mlp_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def build_gnn_mlp_model(gnn_dim, mlp_dim):
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")

    # GNN branch
    gnn_embedding = Dense(64, activation="relu")(gnn_input)
    gnn_embedding = Dense(32, activation="relu")(gnn_embedding)

    # MLP branch
    mlp_embedding = Dense(64, activation="relu")(mlp_input)
    mlp_embedding = Dense(32, activation="relu")(mlp_embedding)

    # Combine
    combined = Concatenate()([gnn_embedding, mlp_embedding])
    f = Dense(64, activation="relu")(combined)
    output = Dense(1, activation="linear", name="gnn_mlp_output")(f)
    
    model = Model(inputs=[gnn_input, mlp_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def build_cnn_gnn_model(patch_shape, gnn_dim):
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")

    # CNN branch
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_embedding = Flatten()(cnn_branch)
    
    # GNN branch
    gnn_embedding = Dense(64, activation="relu")(gnn_input)
    gnn_embedding = Dense(32, activation="relu")(gnn_embedding)

    # Combine
    combined = Concatenate()([cnn_embedding, gnn_embedding])
    f = Dense(128, activation="relu")(combined)
    output = Dense(1, activation="linear", name="cnn_gnn_output")(f)
    
    model = Model(inputs=[cnn_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def build_meta_learner_model():
    # Takes predictions from the 3 base models as input
    pred1_input = Input(shape=(1,), name="pred1_input")
    pred2_input = Input(shape=(1,), name="pred2_input")
    pred3_input = Input(shape=(1,), name="pred3_input")

    # Concatenate the predictions
    combined = Concatenate()([pred1_input, pred2_input, pred3_input])
    
    # Simple MLP as the meta-learner
    f = Dense(32, activation="relu")(combined)
    f = Dense(16, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)
    
    model = Model(inputs=[pred1_input, pred2_input, pred3_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

# ==================== 6. Create Data Generators for Base Models ==================== #
# NOTE: We create generators that provide only the necessary inputs for each base model.
class CNNDropoutGenerator(DataGenerator):
    def __getitem__(self, index):
        (batch_cnn, batch_mlp, batch_gnn), batch_y = super().__getitem__(index)
        return (batch_cnn, batch_mlp), batch_y

class GNNDropoutGenerator(DataGenerator):
    def __getitem__(self, index):
        (batch_cnn, batch_mlp, batch_gnn), batch_y = super().__getitem__(index)
        return (batch_gnn, batch_mlp), batch_y

class MLPDropoutGenerator(DataGenerator):
    def __getitem__(self, index):
        (batch_cnn, batch_mlp, batch_gnn), batch_y = super().__getitem__(index)
        return (batch_cnn, batch_gnn), batch_y

def get_base_model_predictions(model, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size):
    num_samples = len(y)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords[i:i+batch_size]
        batch_mlp = mlp_data[i:i+batch_size]
        batch_gnn = gnn_data[i:i+batch_size, :]
        
        batch_cnn = extract_patch_for_generator(
            batch_coords, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
        )
        
        # Check which inputs the model expects and provide them
        input_names = [inp.name for inp in model.inputs]
        input_dict = {}
        if 'cnn_input' in input_names:
            input_dict['cnn_input'] = batch_cnn
        if 'mlp_input' in input_names:
            input_dict['mlp_input'] = batch_mlp
        if 'gnn_input' in input_names:
            input_dict['gnn_input'] = batch_gnn
            
        y_pred_list.append(model.predict(input_dict).flatten())
            
    return np.concatenate(y_pred_list)


# ==================== Run the Analysis ==================== #

# Redirect all print statements to a string (this is for logging to a file).
old_stdout = sys.stdout
sys.stdout = captured_output = StringIO()

print("\n" + "="*80)
print(f"Analyzing Stacked Deep Ensemble for BUFFER_METERS = {BUFFER_METERS}m")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

mlp_input_dim = mlp_train.shape[1]

# --- Train Base Models ---
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

print("\n--- Training CNN-MLP Base Model ---")
cnn_mlp_model = build_cnn_mlp_model(cnn_patch_shape, mlp_input_dim)
cnn_mlp_train_gen = CNNDropoutGenerator(
    coords=coords_train, mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=True
)
cnn_mlp_model.fit(cnn_mlp_train_gen, epochs=100, verbose=1, callbacks=[early_stopping], validation_data=cnn_mlp_train_gen)

print("\n--- Training GNN-MLP Base Model ---")
gnn_mlp_model = build_gnn_mlp_model(gnn_input_dim, mlp_input_dim)
gnn_mlp_train_gen = GNNDropoutGenerator(
    coords=coords_train, mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=True
)
gnn_mlp_model.fit(gnn_mlp_train_gen, epochs=100, verbose=1, callbacks=[early_stopping], validation_data=gnn_mlp_train_gen)

print("\n--- Training CNN-GNN Base Model ---")
cnn_gnn_model = build_cnn_gnn_model(cnn_patch_shape, gnn_input_dim)
cnn_gnn_train_gen = MLPDropoutGenerator(
    coords=coords_train, mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=True
)
cnn_gnn_model.fit(cnn_gnn_train_gen, epochs=100, verbose=1, callbacks=[early_stopping], validation_data=cnn_gnn_train_gen)

# --- Generate predictions for meta-learner ---
# Get predictions from base models on training data
preds1_train = get_base_model_predictions(cnn_mlp_model, coords_train, mlp_train, gnn_train, y_train, raster_paths, BUFFER_METERS, batch_size)
preds2_train = get_base_model_predictions(gnn_mlp_model, coords_train, mlp_train, gnn_train, y_train, raster_paths, BUFFER_METERS, batch_size)
preds3_train = get_base_model_predictions(cnn_gnn_model, coords_train, mlp_train, gnn_train, y_train, raster_paths, BUFFER_METERS, batch_size)

meta_train_inputs = (preds1_train.reshape(-1, 1), preds2_train.reshape(-1, 1), preds3_train.reshape(-1, 1))

# --- Train Meta-Learner ---
print("\n--- Training Meta-Learner Model ---")
meta_model = build_meta_learner_model()
meta_model.fit(meta_train_inputs, y_train, epochs=100, verbose=1, callbacks=[early_stopping], validation_split=0.2)

# --- Get predictions from base models on test data ---
preds1_test = get_base_model_predictions(cnn_mlp_model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, batch_size)
preds2_test = get_base_model_predictions(gnn_mlp_model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, batch_size)
preds3_test = get_base_model_predictions(cnn_gnn_model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, batch_size)

meta_test_inputs = (preds1_test.reshape(-1, 1), preds2_test.reshape(-1, 1), preds3_test.reshape(-1, 1))

# --- Evaluate with Meta-Learner ---
y_pred = meta_model.predict(meta_test_inputs).flatten()
r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n Stacked Deep Ensemble Model Performance ({BUFFER_METERS}m):")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# --- NEW: Feature Importance for Meta-Learner ---
print("\n" + "-"*50)
print(f"Meta-Learner Feature Importance (Permutation-based)")
print("-"*50)
baseline_r2 = r2_test

# Importance for CNN-MLP predictions
preds1_test_shuffled = np.copy(preds1_test)
np.random.shuffle(preds1_test_shuffled)
shuffled_test_inputs = (preds1_test_shuffled.reshape(-1, 1), preds2_test.reshape(-1, 1), preds3_test.reshape(-1, 1))
y_pred_shuffled = meta_model.predict(shuffled_test_inputs).flatten()
r2_shuffled = r2_score(y_test, y_pred_shuffled)
importance_cnn_mlp = baseline_r2 - r2_shuffled
print(f"Importance of CNN-MLP predictions (R² drop): {importance_cnn_mlp:.4f}")

# Importance for GNN-MLP predictions
preds2_test_shuffled = np.copy(preds2_test)
np.random.shuffle(preds2_test_shuffled)
shuffled_test_inputs = (preds1_test.reshape(-1, 1), preds2_test_shuffled.reshape(-1, 1), preds3_test.reshape(-1, 1))
y_pred_shuffled = meta_model.predict(shuffled_test_inputs).flatten()
r2_shuffled = r2_score(y_test, y_pred_shuffled)
importance_gnn_mlp = baseline_r2 - r2_shuffled
print(f"Importance of GNN-MLP predictions (R² drop): {importance_gnn_mlp:.4f}")

# Importance for CNN-GNN predictions
preds3_test_shuffled = np.copy(preds3_test)
np.random.shuffle(preds3_test_shuffled)
shuffled_test_inputs = (preds1_test.reshape(-1, 1), preds2_test.reshape(-1, 1), preds3_test_shuffled.reshape(-1, 1))
y_pred_shuffled = meta_model.predict(shuffled_test_inputs).flatten()
r2_shuffled = r2_score(y_test, y_pred_shuffled)
importance_cnn_gnn = baseline_r2 - r2_shuffled
print(f"Importance of CNN-GNN predictions (R² drop): {importance_cnn_gnn:.4f}")

# ==================== NEW: Save all info to a folder ==================== #
# Restore standard output
sys.stdout = old_stdout
printed_output = captured_output.getvalue()

output_folder = "stacked_ensemble"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save all four models
cnn_mlp_model_path = os.path.join(output_folder, "cnn_mlp_model.keras")
cnn_mlp_model.save(cnn_mlp_model_path)
print(f"CNN-MLP model saved to: {cnn_mlp_model_path}")

gnn_mlp_model_path = os.path.join(output_folder, "gnn_mlp_model.keras")
gnn_mlp_model.save(gnn_mlp_model_path)
print(f"GNN-MLP model saved to: {gnn_mlp_model_path}")

cnn_gnn_model_path = os.path.join(output_folder, "cnn_gnn_model.keras")
cnn_gnn_model.save(cnn_gnn_model_path)
print(f"CNN-GNN model saved to: {cnn_gnn_model_path}")

meta_model_path = os.path.join(output_folder, "meta_learner.keras")
meta_model.save(meta_model_path)
print(f"Meta-learner model saved to: {meta_model_path}")


# Save the base model predictions and true labels
np.save(os.path.join(output_folder, "preds1_train.npy"), preds1_train)
np.save(os.path.join(output_folder, "preds2_train.npy"), preds2_train)
np.save(os.path.join(output_folder, "preds3_train.npy"), preds3_train)
np.save(os.path.join(output_folder, "preds1_test.npy"), preds1_test)
np.save(os.path.join(output_folder, "preds2_test.npy"), preds2_test)
np.save(os.path.join(output_folder, "preds3_test.npy"), preds3_test)
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred.npy"), y_pred)
print(f"Predictions and true labels saved as .npy files.")

# Save the feature importance results
feature_importance = {
    "CNN-MLP_importance": importance_cnn_mlp,
    "GNN-MLP_importance": importance_gnn_mlp,
    "CNN-GNN_importance": importance_cnn_gnn
}
importance_path = os.path.join(output_folder, "meta_learner_importance.pkl")
with open(importance_path, 'wb') as f:
    pickle.dump(feature_importance, f)
print(f"Meta-learner feature importance saved to: {importance_path}")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

print("All information successfully saved.")

# Garbage collect to free up memory now that everything is saved
del cnn_mlp_model, gnn_mlp_model, cnn_gnn_model, meta_model
gc.collect()

20619

# CNN + LSTM (Spatio-Temporal)

- *If* LULC rasters are time-series (2017–2022), stack them and process with **ConvLSTM2D**.

```
Time-Series Rasters → ConvLSTM2D → Flatten → Dense → Fusion → Output

```

- **Fusion:** Combine ConvLSTM output with MLP (hydrology) and GNN (spatial network).

In [48]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ConvLSTM2D, Flatten, Dense, Concatenate, Dropout, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle

# Define the single buffer size to use
BUFFER_METERS = 500
# Define number of time steps for mock data
TIME_STEPS = 5

# ==================== 1. Load Data ==================== #
# NOTE: The data loading logic remains the same as in the original script.
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# NOTE: This code assumes the rasters are not time-series.
# The `generate_mock_time_series` function below will create a time-series
# for demonstration purposes. In a real-world scenario, you would load
# different raster data for each time step.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    if np.nanmax(arr) != 0:
                        arr /= np.nanmax(arr)
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

def generate_mock_time_series(patches, time_steps):
    """
    Generates mock time-series data by stacking the same patch for 'time_steps'
    time steps. In a real-world scenario, you would have different rasters
    for each time step, and this function would not be needed.
    
    Input shape: (batch_size, height, width, channels)
    Output shape: (batch_size, time_steps, height, width, channels)
    """
    return np.stack([patches] * time_steps, axis=1)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, time_steps=TIME_STEPS, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.buffer_meters = buffer_meters
        self.time_steps = time_steps

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )
        
        # Generate mock time-series data
        batch_cnn_time_series = generate_mock_time_series(batch_cnn, self.time_steps)

        return (batch_cnn_time_series, batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define Spatio-Temporal Model ==================== #
def build_spatio_temporal_model(time_series_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=time_series_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- ConvLSTM2D Branch for Spatio-Temporal Data ---
    # `return_sequences=False` means we get the final output of the sequence
    conv_lstm_branch = ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        padding='same',
        return_sequences=False,
        activation='relu'
    )(cnn_input)
    
    # Flatten and get embedding
    cnn_embedding = Flatten()(conv_lstm_branch)
    cnn_embedding = Dense(128, activation="relu", name="cnn_embedding")(cnn_embedding)

    # --- MLP Branch with Embedding ---
    mlp_embedding = Dense(64, activation="relu")(mlp_input)
    mlp_embedding = Dense(32, activation="relu", name="mlp_embedding")(mlp_embedding)

    # --- GNN Branch with Embedding ---
    gnn_embedding = Dense(64, activation="relu")(gnn_input)
    gnn_embedding = Dense(32, activation="relu", name="gnn_embedding")(gnn_embedding)

    # --- Fusion ---
    combined_embedding = Concatenate(name="combined_embedding")([cnn_embedding, mlp_embedding, gnn_embedding])
    
    # Final dense layers for prediction
    f = Dense(128, activation="relu")(combined_embedding)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters, time_steps, batch_size=4):
    """
    Evaluates the model on test data and returns R², RMSE, and predictions.
    """
    num_samples = len(y_test)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords_test[i:i+batch_size]
        batch_mlp = mlp_test[i:i+batch_size]
        batch_gnn = gnn_test[i:i+batch_size, :]
        
        batch_cnn = extract_patch_for_generator(
            batch_coords, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
        )
        batch_cnn_time_series = generate_mock_time_series(batch_cnn, time_steps)
        
        y_pred_list.append(model.predict((batch_cnn_time_series, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return r2, rmse, y_pred

def calculate_mlp_feature_importance(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters, time_steps, numeric_cols, batch_size=4):
    """
    Calculates feature importance for MLP features using a permutation-based approach.
    """
    # First, get baseline performance on the original test set
    _, baseline_rmse, _ = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, buffer_meters, time_steps, batch_size)
    
    feature_importances = {}
    
    # Iterate through each MLP feature
    for i, feature_name in enumerate(numeric_cols):
        print(f"Calculating importance for feature: {feature_name}")
        
        # Create a copy of the MLP test data to shuffle one feature
        shuffled_mlp_test = mlp_test.copy()
        
        # Shuffle the current feature's column
        np.random.shuffle(shuffled_mlp_test[:, i])
        
        # Evaluate model with shuffled data
        _, shuffled_rmse, _ = evaluate_model(model, coords_test, shuffled_mlp_test, gnn_test, y_test, raster_paths, buffer_meters, time_steps, batch_size)
        
        # The importance is the increase in RMSE
        importance = shuffled_rmse - baseline_rmse
        feature_importances[feature_name] = importance
        
    return feature_importances

# ==================== Run the Analysis ==================== #
# Redirect output to a string for later saving
old_stdout = sys.stdout
sys.stdout = captured_output = StringIO()

print("\n" + "="*80)
print(f"Analyzing CNN + LSTM Model with {TIME_STEPS} mock time steps")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    time_series_shape = (TIME_STEPS, patch_width, patch_width, len(raster_paths))

mlp_input_dim = mlp_train.shape[1]

model = build_spatio_temporal_model(time_series_shape, gnn_input_dim, mlp_input_dim)
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train, mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate ==================== #
# Evaluate on training data
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

# Evaluate on test data
r2_test, rmse_test, y_pred_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, TIME_STEPS, batch_size=batch_size)

print(f"\n Spatio-Temporal Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# ==================== 9. Save all info to a folder ==================== #
# Restore standard output
sys.stdout = old_stdout
printed_output = captured_output.getvalue()

output_folder = "cnn_lstm"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save the model
model_path = os.path.join(output_folder, "spatio_temporal_model.keras")
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the predictions and true labels
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred_train.npy"), y_pred_train)
np.save(os.path.join(output_folder, "y_pred_test.npy"), y_pred_test)
print(f"Predictions and true labels saved as .npy files.")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

# ==================== 10. Calculate and save feature importance ==================== #
print("\n" + "="*80)
print("Calculating MLP Feature Importance...")
print("="*80)

mlp_importance = calculate_mlp_feature_importance(
    model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, TIME_STEPS, numeric_cols, batch_size=batch_size
)

# Save feature importance to a pickle file
importance_path = os.path.join(output_folder, "mlp_feature_importance.pkl")
with open(importance_path, 'wb') as f:
    pickle.dump(mlp_importance, f)

print(f"MLP Feature importance saved to: {importance_path}")
print("Feature Importances (change in RMSE):")
for feature, importance in mlp_importance.items():
    print(f"  - {feature}: {importance:.4f}")

print("\nAll information successfully saved.")

# Garbage collect to free up memory now that everything is saved
del model, history, train_generator
gc.collect()

Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 5, 100,    │          0 │ -                 │
│ (InputLayer)        │ 100, 26)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_2       │ (None, 100, 100,  │    207,616 │ cnn_input[0][0]   │
│ (ConvLSTM2D)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_31          │ (None, 640000)    │          0 │ conv_lstm2d_2[0]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_192 (Dense)   │ (None, 64)        │      1,024 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_193 (Dense)   │ (None, 64)        │      7,104 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_embedding       │ (None, 128)       │ 81,920,128 │ flatten_31[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_embedding       │ (None, 32)        │      2,080 │ dense_192[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_embedding       │ (None, 32)        │      2,080 │ dense_193[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combined_embedding  │ (None, 192)       │          0 │ cnn_embedding[0]… │
│ (Concatenate)       │                   │            │ mlp_embedding[0]… │
│                     │                   │            │ gnn_embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_194 (Dense)   │ (None, 128)       │     24,704 │ combined_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_85          │ (None, 128)       │          0 │ dense_194[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_195 (Dense)   │ (None, 64)        │      8,256 │ dropout_85[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_195[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,173,057 (313.47 MB)

 Trainable params: 82,173,057 (313.47 MB)

 Non-trainable params: 0 (0.00 B)

16209

# Transformer-based Fusion (CNN + GNN + MLP)

- **Idea:** Use a **Transformer Encoder** to fuse embeddings from CNN, GNN, and MLP branches.
- **Architecture:**

```
CNN Embedding ┐
GNN Embedding ├── Transformer Encoder → Dense → Output
MLP Embedding ┘

```

In [49]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print(f"Using {len(raster_paths)} raster layers for CNN input.")
for r in raster_paths:
    print("  -", os.path.basename(r))

# ==================== 3. Create a Custom Data Generator ==================== #
def extract_patch_for_generator(coords, raster_files, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height):
    """
    Extracts a batch of patches from rasters for a given set of coordinates.
    This function is optimized to be called by the data generator for each batch.
    """
    patches = []
    # Loop through each coordinate pair in the batch
    for lon, lat in coords:
        channels = []
        # Loop through each raster file to get a single patch for each raster
        for rfile in raster_files:
            with rasterio.open(rfile) as src:
                try:
                    row, col = src.index(lon, lat)
                    win = Window(col - buffer_pixels_x, row - buffer_pixels_y, patch_width, patch_height)
                    arr = src.read(1, window=win, boundless=True, fill_value=0)
                    arr = arr.astype(np.float32)

                    if np.nanmax(arr) != 0:
                        arr /= np.nanmax(arr)
                except Exception as e:
                    print(f"Error processing {rfile} for coordinates ({lon}, {lat}): {e}")
                    arr = np.zeros((patch_width, patch_height), dtype=np.float32)
            channels.append(arr)
        patches.append(np.stack(channels, axis=-1))
    
    return np.array(patches)

class DataGenerator(Sequence):
    def __init__(self, coords, mlp_data, gnn_data, y, raster_paths, buffer_meters, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.coords = coords
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.raster_paths = raster_paths
        self.buffer_meters = buffer_meters
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))

        # Pre-calculate patch size from the first raster
        with rasterio.open(raster_paths[0]) as src:
            res_x, res_y = src.res
            self.buffer_pixels_x = int(self.buffer_meters / res_x)
            self.buffer_pixels_y = int(self.buffer_meters / res_y)
            self.patch_width = 2 * self.buffer_pixels_x
            self.patch_height = 2 * self.buffer_pixels_y

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_coords = self.coords[batch_indices]
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]

        # Extract CNN patches for the current batch
        batch_cnn = extract_patch_for_generator(
            batch_coords,
            self.raster_paths,
            self.buffer_pixels_x,
            self.buffer_pixels_y,
            self.patch_width,
            self.patch_height
        )
        
        return (batch_cnn, batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define Transformer-based Fusion Model ==================== #
def build_transformer_fusion_model(patch_shape, gnn_dim, mlp_dim):
    # Inputs for all branches
    cnn_input = Input(shape=patch_shape, name="cnn_input")
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- CNN Branch ---
    cnn_branch = Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_branch = Conv2D(64, (3,3), activation="relu", padding="same")(cnn_branch)
    cnn_branch = MaxPooling2D((2,2))(cnn_branch)
    cnn_embedding = Flatten(name="cnn_embedding_flatten")(cnn_branch)
    
    # --- MLP Branch ---
    mlp_embedding = Dense(128, activation="relu")(mlp_input)
    mlp_embedding = Dense(64, activation="relu", name="mlp_embedding")(mlp_embedding)

    # --- GNN Branch ---
    gnn_embedding = Dense(128, activation="relu")(gnn_input)
    gnn_embedding = Dense(64, activation="relu", name="gnn_embedding")(gnn_embedding)

    # --- Transformer Fusion ---
    # To feed into the transformer, we need to make all embeddings have the same dimension.
    # Let's use a dense layer to project them to a common size.
    projection_dim = 64
    cnn_proj = Dense(projection_dim)(cnn_embedding)
    mlp_proj = Dense(projection_dim)(mlp_embedding)
    gnn_proj = Dense(projection_dim)(gnn_embedding)

    # Stack the embeddings to create a sequence for the transformer
    # Shape becomes (None, 3, projection_dim)
    # Corrected code to use Keras-compatible operations
    cnn_expanded = Reshape((1, projection_dim))(cnn_proj)
    mlp_expanded = Reshape((1, projection_dim))(mlp_proj)
    gnn_expanded = Reshape((1, projection_dim))(gnn_proj)
    embeddings = Concatenate(axis=1)([cnn_expanded, mlp_expanded, gnn_expanded])

    # Transformer Encoder block
    transformer_output = MultiHeadAttention(
        num_heads=4,
        key_dim=projection_dim
    )(embeddings, embeddings)
    transformer_output = Dropout(0.2)(transformer_output)
    transformer_output = LayerNormalization(epsilon=1e-6)(embeddings + transformer_output)
    
    # The output from the transformer is a sequence of 3 vectors.
    # We flatten this for the final prediction layer.
    transformer_output_flattened = Flatten()(transformer_output)
    
    # Final dense layers for prediction
    f = Dense(128, activation="relu")(transformer_output_flattened)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[cnn_input, mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, coords, mlp_data, gnn_data, y_true, raster_paths, buffer_meters, batch_size=4, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, and predictions.
    """
    num_samples = len(y_true)
    y_pred_list = []
    
    with rasterio.open(raster_paths[0]) as src:
        res_x, res_y = src.res
        buffer_pixels_x = int(buffer_meters / res_x)
        buffer_pixels_y = int(buffer_meters / res_y)
        patch_width = 2 * buffer_pixels_x
        patch_height = 2 * buffer_pixels_y

    for i in range(0, num_samples, batch_size):
        batch_coords = coords[i:i+batch_size]
        batch_mlp = mlp_data[i:i+batch_size]
        batch_gnn = gnn_data[i:i+batch_size, :]
        
        batch_cnn = extract_patch_for_generator(
            batch_coords, raster_paths, buffer_pixels_x, buffer_pixels_y, patch_width, patch_height
        )
        
        y_pred_list.append(model.predict((batch_cnn, batch_mlp, batch_gnn)).flatten())
        
    y_pred = np.concatenate(y_pred_list)
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        return r2, rmse

def calculate_permutation_importance(model, coords, mlp_data, gnn_data, y_true, raster_paths, buffer_meters, batch_size=4):
    """
    Calculates permutation feature importance for the three model branches.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _ = evaluate_model(model, coords, mlp_data, gnn_data, y_true, raster_paths, buffer_meters, batch_size=batch_size)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute CNN input
    shuffled_indices = np.random.permutation(len(y_true))
    coords_shuffled = coords[shuffled_indices]
    shuffled_r2, _ = evaluate_model(model, coords_shuffled, mlp_data, gnn_data, y_true, raster_paths, buffer_meters, batch_size=batch_size)
    importance['CNN'] = baseline_r2 - shuffled_r2
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _ = evaluate_model(model, coords, shuffled_mlp_data, gnn_data, y_true, raster_paths, buffer_meters, batch_size=batch_size)
    importance['MLP'] = baseline_r2 - shuffled_r2

    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _ = evaluate_model(model, coords, mlp_data, shuffled_gnn_data, y_true, raster_paths, buffer_meters, batch_size=batch_size)
    importance['GNN'] = baseline_r2 - shuffled_r2

    return importance

# ==================== Run the Analysis ==================== #
# Redirect output to a string for later saving
old_stdout = sys.stdout
sys.stdout = captured_output = StringIO()

print("\n" + "="*80)
print(f"Analyzing Transformer-based Fusion Model for BUFFER_METERS = {BUFFER_METERS}m")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)

# Calculate CNN patch shape based on the current buffer size
with rasterio.open(raster_paths[0]) as src:
    res_x, res_y = src.res
    buffer_pixels_x = int(BUFFER_METERS / res_x)
    patch_width = 2 * buffer_pixels_x
    cnn_patch_shape = (patch_width, patch_width, len(raster_paths))

mlp_input_dim = mlp_train.shape[1]

model = build_transformer_fusion_model(cnn_patch_shape, gnn_input_dim, mlp_input_dim)
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    coords=coords_train, mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    raster_paths=raster_paths, buffer_meters=BUFFER_METERS, batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate & Perform Feature Importance ==================== #
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

r2_test, rmse_test = evaluate_model(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, batch_size=batch_size)

print(f"\n Transformer-based Fusion Model Performance ({BUFFER_METERS}m):")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# Calculate and print feature importance
feature_importance = calculate_permutation_importance(model, coords_test, mlp_test, gnn_test, y_test, raster_paths, BUFFER_METERS, batch_size=batch_size)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")


# ==================== 9. Save all info to a folder ==================== #
# Restore standard output
sys.stdout = old_stdout
printed_output = captured_output.getvalue()

output_folder = "transformer_fusion"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save the model
model_path = os.path.join(output_folder, "transformer_fusion_model.keras")
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the predictions and true labels
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred_train.npy"), y_pred_train)
np.save(os.path.join(output_folder, "y_pred_test.npy"), y_pred_test)
print(f"Predictions and true labels saved as .npy files.")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

# Save the feature importance dictionary as a .pkl file
importance_path = os.path.join(output_folder, "feature_importance.pkl")
with open(importance_path, 'wb') as f:
    pickle.dump(feature_importance, f)
print(f"Feature importance results saved to: {importance_path}")

print("\nAll information successfully saved.")

# Garbage collect to free up memory now that everything is saved
del model, history, train_generator
gc.collect()

Model: "functional_48"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_input           │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 26)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 100, 100,  │      7,520 │ cnn_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_58    │ (None, 50, 50,    │          0 │ conv2d_62[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_63 (Conv2D)  │ (None, 50, 50,    │     18,496 │ max_pooling2d_58… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_59    │ (None, 25, 25,    │          0 │ conv2d_63[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_196 (Dense)   │ (None, 128)       │      2,048 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_197 (Dense)   │ (None, 128)       │     14,208 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_embedding_flat… │ (None, 40000)     │          0 │ max_pooling2d_59… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_embedding       │ (None, 64)        │      8,256 │ dense_196[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_embedding       │ (None, 64)        │      8,256 │ dense_197[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_198 (Dense)   │ (None, 64)        │  2,560,064 │ cnn_embedding_fl… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_199 (Dense)   │ (None, 64)        │      4,160 │ mlp_embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_200 (Dense)   │ (None, 64)        │      4,160 │ gnn_embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_7 (Reshape) │ (None, 1, 64)     │          0 │ dense_198[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_8 (Reshape) │ (None, 1, 64)     │          0 │ dense_199[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_9 (Reshape) │ (None, 1, 64)     │          0 │ dense_200[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, 3, 64)     │          0 │ reshape_7[0][0],  │
│ (Concatenate)       │                   │            │ reshape_8[0][0],  │
│                     │                   │            │ reshape_9[0][0] 

 Total params: 2,726,689 (10.40 MB)

 Trainable params: 2,726,689 (10.40 MB)

 Non-trainable params: 0 (0.00 B)

14991

# GNN-MLP

In [50]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print("Note: Raster data is not used in this GNN-MLP model.")

# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define GNN-MLP Fusion Model ==================== #
def build_gnn_mlp_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- MLP Branch ---
    mlp_embedding = Dense(128, activation="relu")(mlp_input)
    mlp_embedding = Dense(64, activation="relu", name="mlp_embedding")(mlp_embedding)

    # --- GNN Branch ---
    gnn_embedding = Dense(128, activation="relu")(gnn_input)
    gnn_embedding = Dense(64, activation="relu", name="gnn_embedding")(gnn_embedding)

    # --- Concatenate Embeddings ---
    combined = Concatenate()([mlp_embedding, gnn_embedding])
    
    # Final dense layers for prediction
    f = Dense(128, activation="relu")(combined)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, and predictions.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

def calculate_permutation_importance(model, mlp_data, gnn_data, y_true):
    """
    Calculates permutation feature importance for the MLP and GNN branches.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP'] = baseline_r2 - shuffled_r2

    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN'] = baseline_r2 - shuffled_r2

    return importance
        
# ==================== Run the Analysis ==================== #
# Redirect output to a string for later saving
old_stdout = sys.stdout
sys.stdout = captured_output = StringIO()

print("\n" + "="*80)
print(f"Analyzing GNN-MLP Fusion Model")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

model = build_gnn_mlp_model(mlp_input_dim, gnn_input_dim)
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate & Perform Feature Importance ==================== #
# Predict on the training data using the generator
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

# Evaluate on the test data using the updated function
r2_test, rmse_test = evaluate_model(model, mlp_test, gnn_test, y_test)
y_pred_test = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)

print(f"\n GNN-MLP Fusion Model Performance:")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# Calculate and print feature importance
feature_importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")

# ==================== 9. Save all info to a folder ==================== #
# Restore standard output
sys.stdout = old_stdout
printed_output = captured_output.getvalue()

output_folder = "gnn_mlp"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save the model
model_path = os.path.join(output_folder, "gnn_mlp_model.keras")
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the predictions and true labels
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred_train.npy"), y_pred_train)
np.save(os.path.join(output_folder, "y_pred_test.npy"), y_pred_test)
print(f"Predictions and true labels saved as .npy files.")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

# Save the feature importance dictionary as a .pkl file
importance_path = os.path.join(output_folder, "feature_importance.pkl")
with open(importance_path, 'wb') as f:
    pickle.dump(feature_importance, f)
print(f"Feature importance results saved to: {importance_path}")

print("\nAll information successfully saved.")

# Garbage collect to free up memory now that everything is saved
del model, history, train_generator
gc.collect()

Model: "functional_49"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_203 (Dense)   │ (None, 128)       │      2,048 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_204 (Dense)   │ (None, 128)       │     14,208 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_embedding       │ (None, 64)        │      8,256 │ dense_203[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_embedding       │ (None, 64)        │      8,256 │ dense_204[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_29      │ (None, 128)       │          0 │ mlp_embedding[0]… │
│ (Concatenate)       │                   │            │ gnn_embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_205 (Dense)   │ (None, 128)       │     16,512 │ concatenate_29[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_89          │ (None, 128)       │          0 │ dense_205[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_206 (Dense)   │ (None, 64)        │      8,256 │ dropout_89[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_206[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,601 (225.00 KB)

 Trainable params: 57,601 (225.00 KB)

 Non-trainable params: 0 (0.00 B)

14845

# GNN-MLP Autoencoder

In [51]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print("Note: Raster data is not used in this GNN-MLP model.")

# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define GNN-MLP Fusion Autoencoder Model ==================== #
def build_gnn_mlp_autoencoder_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Encoder Branch (MLP) ---
    mlp_encoded = Dense(128, activation="relu")(mlp_input)
    mlp_encoded = Dense(64, activation="relu", name="mlp_encoder_output")(mlp_encoded)

    # --- Encoder Branch (GNN) ---
    gnn_encoded = Dense(128, activation="relu")(gnn_input)
    gnn_encoded = Dense(64, activation="relu", name="gnn_encoder_output")(gnn_encoded)

    # --- Bottleneck/Latent Space ---
    # Concatenate the encoded representations
    latent_space = Concatenate(name="latent_space")([mlp_encoded, gnn_encoded])
    
    # --- Decoder Branch for Prediction ---
    # The decoder takes the latent space and performs the final prediction
    f = Dense(128, activation="relu")(latent_space)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, and predictions.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

def calculate_permutation_importance(model, mlp_data, gnn_data, y_true):
    """
    Calculates permutation feature importance for the MLP and GNN branches.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP'] = baseline_r2 - shuffled_r2

    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN'] = baseline_r2 - shuffled_r2

    return importance
        
# ==================== Run the Analysis ==================== #
# Redirect output to a string for later saving
old_stdout = sys.stdout
sys.stdout = captured_output = StringIO()

print("\n" + "="*80)
print(f"Analyzing GNN-MLP Autoencoder Model")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

model = build_gnn_mlp_autoencoder_model(mlp_input_dim, gnn_input_dim)
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate & Perform Feature Importance ==================== #
# Predict on the training data using the generator
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

# Evaluate on the test data using the updated function
r2_test, rmse_test = evaluate_model(model, mlp_test, gnn_test, y_test)
y_pred_test = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)

print(f"\n GNN-MLP Autoencoder Model Performance:")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# Calculate and print feature importance
feature_importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")

# ==================== 9. Save all info to a folder ==================== #
# Restore standard output
sys.stdout = old_stdout
printed_output = captured_output.getvalue()

output_folder = "gnn_mlp_ae"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save the model
model_path = os.path.join(output_folder, "gnn_mlp_ae_model.keras")
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the predictions and true labels
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred_train.npy"), y_pred_train)
np.save(os.path.join(output_folder, "y_pred_test.npy"), y_pred_test)
print(f"Predictions and true labels saved as .npy files.")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

# Save the feature importance dictionary as a .pkl file
importance_path = os.path.join(output_folder, "feature_importance.pkl")
with open(importance_path, 'wb') as f:
    pickle.dump(feature_importance, f)
print(f"Feature importance results saved to: {importance_path}")

print("\nAll information successfully saved.")

# Garbage collect to free up memory now that everything is saved
del model, history, train_generator
gc.collect()

Model: "functional_50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_207 (Dense)   │ (None, 128)       │      2,048 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_208 (Dense)   │ (None, 128)       │     14,208 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_encoder_output  │ (None, 64)        │      8,256 │ dense_207[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_encoder_output  │ (None, 64)        │      8,256 │ dense_208[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_space        │ (None, 128)       │          0 │ mlp_encoder_outp… │
│ (Concatenate)       │                   │            │ gnn_encoder_outp… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_209 (Dense)   │ (None, 128)       │     16,512 │ latent_space[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_90          │ (None, 128)       │          0 │ dense_209[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_210 (Dense)   │ (None, 64)        │      8,256 │ dropout_90[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_210[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,601 (225.00 KB)

 Trainable params: 57,601 (225.00 KB)

 Non-trainable params: 0 (0.00 B)

8731

# GCN GAT

In [52]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import gc # Import garbage collector
import pickle # Import for saving feature importance results
import sys # Import sys for stdout redirection

# Define the single buffer size to use
BUFFER_METERS = 500

# Save the original stdout
original_stdout = sys.stdout

# Try-finally block to ensure stdout is restored even if errors occur
try:
    # Open the file and redirect stdout to it
    with open('gcn_gat/analysis_output.txt', 'w') as f:
        sys.stdout = f

        # ==================== 1. Load Data ==================== #
        # NOTE: The file paths are relative to the notebook's execution directory.
        # Please ensure they are correct for your environment.
        orig = pd.read_csv("../../data/RainySeason.csv")
        river_100 = pd.read_csv("../data/Samples_100.csv")

        drop_cols = ['Stations','River','Lat','Long','geometry']
        numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

        # Train-test split
        train_orig = orig.sample(10, random_state=42)
        test_orig = orig.drop(train_orig.index)
        train_combined = pd.concat([river_100, train_orig], ignore_index=True)

        # ==================== 2. Collect ALL Rasters ==================== #
        # We are not using rasters in this GNN model, but the paths are still
        # defined for consistency with previous versions.
        raster_paths = []
        raster_paths += glob.glob("../CalIndices/*.tif")
        raster_paths += glob.glob("../LULCMerged/*.tif")
        raster_paths += glob.glob("../IDW/*.tif")

        print("Note: Raster data is not used in this Stacking GNN ensemble model.")

        # ==================== 3. Prepare GNN & MLP Input (only once) ==================== #
        # Split the combined training data into a training and a validation set
        mlp_train_val, mlp_test = train_test_split(train_combined, test_size=len(test_orig), random_state=42)
        y_train_val, y_test = train_test_split(train_combined['RI'], test_size=len(test_orig), random_state=42)
        mlp_train, mlp_val, y_train, y_val = train_test_split(mlp_train_val, y_train_val, test_size=0.2, random_state=42)

        # Now, re-do the distance matrices and scaling with the new splits
        coords_train = mlp_train[['Long', 'Lat']].values
        coords_val = mlp_val[['Long', 'Lat']].values
        coords_test = test_orig[['Long', 'Lat']].values

        # Create distance matrices, which serve as the adjacency matrix for the GNN
        dist_mat_train = distance_matrix(coords_train, coords_train)
        gnn_train_data = np.exp(-dist_mat_train/10) # Using a radial basis function kernel
        dist_mat_val = distance_matrix(coords_val, coords_val)
        gnn_val_data = np.exp(-dist_mat_val/10)
        dist_mat_test = distance_matrix(coords_test, coords_test)
        gnn_test_data = np.exp(-dist_mat_test/10)

        # Scale the MLP features using StandardScaler
        scaler = StandardScaler()
        mlp_train_scaled = scaler.fit_transform(mlp_train[numeric_cols])
        mlp_val_scaled = scaler.transform(mlp_val[numeric_cols])
        mlp_test_scaled = scaler.transform(test_orig[numeric_cols])

        # Convert target data to numpy arrays
        y_train_arr = y_train.values
        y_val_arr = y_val.values
        y_test_arr = y_test.values

        # Add a batch dimension to the data since we're using full-graph training
        mlp_train_data = np.expand_dims(mlp_train_scaled, axis=0)
        gnn_train_data = np.expand_dims(gnn_train_data, axis=0)
        mlp_val_data = np.expand_dims(mlp_val_scaled, axis=0)
        gnn_val_data = np.expand_dims(gnn_val_data, axis=0)
        mlp_test_data = np.expand_dims(mlp_test_scaled, axis=0)
        gnn_test_data = np.expand_dims(gnn_test_data, axis=0)


        # ==================== 4. Define Stacking GNN Ensemble Model ==================== #

        class GCNLayer(Layer):
            """
            Custom GCN Layer. Given the pre-computed similarity matrix, this layer
            aggregates information from neighboring nodes and transforms it.
            """
            def __init__(self, units, activation="relu", **kwargs):
                super(GCNLayer, self).__init__(**kwargs)
                self.units = units
                self.activation = tf.keras.activations.get(activation)

            def build(self, input_shape):
                # input_shape is a list of two shapes: [(batch, nodes, features), (batch, nodes, nodes)]
                mlp_shape, gnn_shape = input_shape
                self.kernel = self.add_weight(
                    shape=(mlp_shape[-1], self.units),
                    initializer="glorot_uniform",
                    trainable=True
                )
                super(GCNLayer, self).build(input_shape)

            def call(self, inputs):
                mlp_input, gnn_input = inputs
                # Perform batched matrix multiplication: (B, N, N) x (B, N, F) -> (B, N, F)
                aggregated_features = tf.matmul(gnn_input, mlp_input)
                # Apply the linear transformation: (B, N, F) x (F, U) -> (B, N, U)
                output = tf.matmul(aggregated_features, self.kernel)
                # Apply activation
                return self.activation(output)

        class GATLayer(Layer):
            """
            Custom GAT Layer. This layer computes attention scores for neighboring
            nodes and aggregates features based on these scores.
            """
            def __init__(self, units, num_heads=4, activation="relu", **kwargs):
                super(GATLayer, self).__init__(**kwargs)
                self.units = units
                self.num_heads = num_heads
                self.activation = tf.keras.activations.get(activation)
                
            def build(self, input_shape):
                mlp_shape, gnn_shape = input_shape
                # The feature transformation kernel
                self.kernel_f = self.add_weight(
                    shape=(mlp_shape[-1], self.units * self.num_heads),
                    initializer="glorot_uniform",
                    trainable=True
                )
                # The attention score kernels
                # Kernel 1 for the source node, Kernel 2 for the target node
                self.kernel_a_1 = self.add_weight(
                    shape=(self.units, 1),
                    initializer="glorot_uniform",
                    trainable=True
                )
                self.kernel_a_2 = self.add_weight(
                    shape=(self.units, 1),
                    initializer="glorot_uniform",
                    trainable=True
                )
                super(GATLayer, self).build(input_shape)
            
            def call(self, inputs):
                mlp_input, gnn_input = inputs
                
                # Linear transformation
                features = tf.matmul(mlp_input, self.kernel_f)
                
                # Split features into attention heads and transpose
                # Shape: (batch_size, num_nodes, num_heads, units)
                features_heads = tf.reshape(features, (-1, tf.shape(mlp_input)[1], self.num_heads, self.units))
                # Transpose to (batch_size, num_heads, num_nodes, units) for easier batched operations
                features_heads_t = tf.transpose(features_heads, perm=[0, 2, 1, 3])
                
                # Calculate attention scores for each head
                # This will be of shape (batch_size, num_heads, num_nodes, 1)
                e_input_1 = tf.matmul(features_heads_t, self.kernel_a_1)
                # This will be of shape (batch_size, num_heads, num_nodes, 1)
                e_input_2_pre_t = tf.matmul(features_heads_t, self.kernel_a_2)
                # Transpose the last two dimensions to get shape (batch_size, num_heads, 1, num_nodes)
                e_input_2 = tf.transpose(e_input_2_pre_t, perm=[0, 1, 3, 2])
                
                # Combine the scores using broadcasting to create the attention matrix for each head
                # Shape will be (batch_size, num_heads, num_nodes, num_nodes)
                e = e_input_1 + e_input_2
                e = tf.nn.leaky_relu(e, alpha=0.2)

                # Mask attention scores for non-existent edges
                # The gnn_input is (batch, nodes, nodes), expand it to (batch, 1, nodes, nodes)
                # so it can be broadcast to the attention_scores shape
                mask = -1e9 * (1.0 - tf.expand_dims(gnn_input, axis=1))
                attention_scores = e + mask
                
                # Softmax normalization across nodes (the last axis)
                attention = tf.nn.softmax(attention_scores, axis=-1)
                
                # Aggregate features
                # Perform batched matrix multiplication: attention (B,H,N,N) * features_heads_t (B,H,N,U) -> (B, H, N, U)
                aggregated_features = tf.matmul(attention, features_heads_t)
                
                # Concatenate heads and apply final activation
                # Reshape to (batch_size, num_nodes, num_heads * units)
                output = tf.reshape(aggregated_features, (-1, tf.shape(mlp_input)[1], self.units * self.num_heads))
                return self.activation(output)

        def build_stacking_ensemble_model(mlp_dim):
            """
            Builds a stacking ensemble model with GCN and GAT base learners
            and an MLP meta-learner.
            
            NOTE: The model architecture has been updated to produce a prediction
            for each node in the graph, rather than a single prediction for the
            entire graph, which was the cause of the previous ValueError.
            """
            # Define inputs for all branches
            # The `None` allows for a variable number of nodes per graph
            mlp_input = Input(shape=(None, mlp_dim), name="mlp_input")
            gnn_input = Input(shape=(None, None), name="gnn_input")
            
            # --- GCN Base Learner Branch ---
            # This branch now outputs node-level features, not a single pooled vector
            gcn_branch = GCNLayer(128, name="gcn_layer_1")([mlp_input, gnn_input])
            gcn_branch = Dropout(0.2)(gcn_branch)
            gcn_output_features = GCNLayer(64, name="gcn_layer_2")([gcn_branch, gnn_input])
            
            # --- GAT Base Learner Branch ---
            # This branch also outputs node-level features
            gat_branch = GATLayer(64, num_heads=4, name="gat_layer_1")([mlp_input, gnn_input])
            gat_branch = Dropout(0.2)(gat_branch)
            gat_output_features = GATLayer(32, num_heads=4, name="gat_layer_2")([gat_branch, gnn_input])
            
            # --- MLP Meta-Learner (now a prediction head) ---
            # Concatenate the node-level feature outputs from the GNN branches
            meta_learner_input = Concatenate(name="meta_learner_input")([gcn_output_features, gat_output_features])
            
            # The final prediction layers operate on the node features to produce a single
            # value for each node.
            meta_learner_output = Dense(16, activation="relu", name="meta_dense_1")(meta_learner_input)
            meta_learner_output = Dense(8, activation="relu", name="meta_dense_2")(meta_learner_output)
            
            # Final prediction layer: one output per node
            final_output = Dense(1, activation="linear", name="final_output")(meta_learner_output)

            # Build and compile the model
            model = Model(inputs=[mlp_input, gnn_input], outputs=final_output)
            model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
            return model

        # Function to calculate permutation feature importance
        def calculate_permutation_importance(model, mlp_data, gnn_data, y_true, feature_names):
            """
            Calculates permutation feature importance for each feature.
            
            NOTE: This function now expects un-batched `mlp_data` and `gnn_data` and
            handles the batching internally for predictions.
            """
            print("\nCalculating permutation feature importance...")
            
            # 1. Calculate a baseline score on the un-permuted data
            # Add batch dimension to data for prediction
            y_pred_baseline = model.predict([np.expand_dims(mlp_data, axis=0), np.expand_dims(gnn_data, axis=0)]).flatten()
            baseline_score = mean_squared_error(y_true, y_pred_baseline)
            
            importance_scores = {}
            
            # 2. Iterate through each feature
            for i, feature in enumerate(feature_names):
                # Create a copy of the data to avoid modifying the original
                X_mlp_permuted = mlp_data.copy()
                
                # Shuffle the values of the current feature
                X_mlp_permuted[:, i] = np.random.permutation(X_mlp_permuted[:, i])
                
                # 3. Make predictions with the permuted data
                # Add batch dimension to permuted data for prediction
                y_pred_permuted = model.predict([np.expand_dims(X_mlp_permuted, axis=0), np.expand_dims(gnn_data, axis=0)]).flatten()
                
                # 4. Calculate the new score and the importance
                permuted_score = mean_squared_error(y_true, y_pred_permuted)
                importance = permuted_score - baseline_score
                
                importance_scores[feature] = importance
                print(f"  Feature '{feature}': Importance = {importance:.4f}")
                
            return importance_scores


        # ==================== Run the Analysis ==================== #
        print("\n" + "="*80)
        print(f"Analyzing Stacking GNN Ensemble Model")
        print("="*80)

        mlp_input_dim = mlp_train_scaled.shape[1]

        # Build the stacking ensemble model
        model = build_stacking_ensemble_model(mlp_input_dim)
        model.summary(print_fn=print) # Use print_fn to redirect summary output

        # ==================== 5. Train Model ==================== #
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        # NOTE: Training on the full graph, not a generator.
        # The y data now also needs a batch dimension to match the model's output
        history = model.fit(
            x=[mlp_train_data, gnn_train_data],
            y=np.expand_dims(y_train_arr, axis=0),
            epochs=100, # Increased epochs for better training
            verbose=1,
            callbacks=[early_stopping],
            validation_data=([mlp_val_data, gnn_val_data], np.expand_dims(y_val_arr, axis=0))
        )

        # ==================== 6. Evaluate ==================== #
        # Predict on the training data
        # The model now outputs predictions for each node, so flattening works correctly.
        y_pred_train = model.predict([mlp_train_data, gnn_train_data]).flatten()
        r2_train = r2_score(y_train_arr, y_pred_train)
        rmse_train = np.sqrt(mean_squared_error(y_train_arr, y_pred_train))

        # Evaluate on the validation data
        y_pred_val = model.predict([mlp_val_data, gnn_val_data]).flatten()
        r2_val = r2_score(y_val_arr, y_pred_val)
        rmse_val = np.sqrt(mean_squared_error(y_val_arr, y_pred_val))

        # Evaluate on the test data
        y_pred_test = model.predict([mlp_test_data, gnn_test_data]).flatten()
        r2_test = r2_score(y_test_arr, y_pred_test)
        rmse_test = np.sqrt(mean_squared_error(y_test_arr, y_pred_test))


        print(f"\n Stacking GNN Ensemble Model Performance:")
        print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
        print(f"R² Test: {r2_val:.4f} | RMSE Test: {rmse_val:.4f}")
        

        # ==================== 7. Permutation Importance and Saving Results ==================== #

        # First, calculate feature importance on the test data
        # Pass un-batched adjacency matrix for consistency
        importance_results = calculate_permutation_importance(
            model=model,
            mlp_data=mlp_test_scaled, # Pass the un-batched data here
            gnn_data=dist_mat_test, # Pass the un-batched adjacency matrix
            y_true=y_test_arr,
            feature_names=numeric_cols
        )

        # Create the directory if it doesn't exist
        output_dir = 'gcn_gat'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save the model in the .keras format
        model_path = os.path.join(output_dir, 'gcn_gat.keras')
        model.save(model_path)
        print(f"\nModel saved to {model_path}")

        # Save the feature importance results as a pickled file
        importance_path = os.path.join(output_dir, 'feature_importance.pkl')
        with open(importance_path, 'wb') as f:
            pickle.dump(importance_results, f)
        print(f"Feature importance results saved to {importance_path}")

        # Save the data splits for reproducibility in .npy format
        np.save(os.path.join(output_dir, 'mlp_train_data.npy'), mlp_train_scaled)
        np.save(os.path.join(output_dir, 'mlp_val_data.npy'), mlp_val_scaled)
        np.save(os.path.join(output_dir, 'mlp_test_data.npy'), mlp_test_scaled)
        np.save(os.path.join(output_dir, 'y_train_data.npy'), y_train_arr)
        np.save(os.path.join(output_dir, 'y_val_data.npy'), y_val_arr)
        np.save(os.path.join(output_dir, 'y_test_data.npy'), y_test_arr)
        print("Training, validation, and test data splits saved to the gnn_gat folder in .npy format.")

        # Garbage collect to free up memory
        del model, history
        gc.collect()

        print("\nAnalysis complete and files have been saved.")

finally:
    # Restore stdout to the original value
    sys.stdout = original_stdout

# Graphsage GAT